# Importing Dependencies

### the notebook in a nutshell 
- importing 
software dependencies we need
- load_data (please read its description it supposed to be very easy to deal with it)

###  preprocesssing 
- I stored some important variables, then I used log transformation for long tailed distributions

- iterative imputer just like what `MAAB` Advised

- transormed the categorical features to be from the string type just to avoid some fitting isssues cause by the 
OneHotEncoder.

- droped the `participant_ID` after storing it for stratification

- I scaled quantative data and also FMRI data 

### modeling
- base model is logisitc regression with l2 regulization (fancy name for punishing the model to stop overfitting to noise) 
- wrapped inside a multioutput classifier that just gives it the ability to predict 2 targets simultaneously
### validation 
- 5 splits stratifiedKFold works fine no need for grouping as participants are unique 
- regular stratified  = 0.5901173137436236
- RepeatedStratifiedKFold =0.5865925294336686 with 7 minutese delay

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score  
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from scipy.stats import hmean
from sklearn.multioutput import MultiOutputClassifier
import scipy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer #it might not work directly if not try the following code line

from sklearn.model_selection import StratifiedGroupKFold,StratifiedKFold,RepeatedStratifiedKFold
from pathlib import Path
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

# Reading Data

In [38]:
"""
just drop the path of your new data directory nothing more than that. 

"""
path = r"C:\Users\Maab\Desktop\ADHD_Kaggle_Competition\Repo\WiDS-Datathon-2025\Data\raw"

def read_data(base_path:str) -> pd.DataFrame :
    path = Path(base_path)
    trc=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_CATEGORICAL_METADATA_new.xlsx')
    trq=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_QUANTITATIVE_METADATA_new.xlsx')
    trf=pd.read_csv(path   /'TRAIN_NEW'  / 'TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
    trs=pd.read_excel(path /'TRAIN_NEW'  / 'TRAINING_SOLUTIONS.xlsx')  
    tsc=pd.read_excel(path /'TEST'      / 'TEST_CATEGORICAL.xlsx')
    tsq=pd.read_excel(path /'TEST'       / 'TEST_QUANTITATIVE_METADATA.xlsx')    
    tsf=pd.read_csv(path   /'TEST'       / 'TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')    
    sub=pd.read_excel(path / 'SAMPLE_SUBMISSION.xlsx')
    dic=pd.read_excel(path /'Data Dictionary.xlsx')
    return trc, trq, trf, trs, tsc, tsq, tsf, sub, dic

trc, trq, trf, trs, tsc, tsq, tsf, sub, dic = read_data(base_path=path)

# Data Merging 
cq = pd.merge(trc, trq, on='participant_id', how='left')
feat = pd.merge(cq, trf, on='participant_id', how='left')  
qc = pd.merge(tsc, tsq, on='participant_id', how='left')
train = pd.merge(feat, trs, on='participant_id', how='left') 
test = pd.merge(qc, tsf, on='participant_id', how='left')

# Adding Features before Preprocessing

In [39]:
# Your coefficients (estimated from your description)
female_coefs = {
    'ColorVision_CV_Score': 2.900,
    'SDQ_SDQ_Emotional_Problems': 2.814,
    'SDQ_SDQ_Prosocial': 2.661,
    'SDQ_SDQ_Internalizing': 1.382,
    'SDQ_SDQ_Conduct_Problems': 1.329,
    'SDQ_SDQ_Generating_Impact': 1.109,
    'APQ_P_APQ_P_PM': 0.642
}

male_coefs = {
    'SDQ_SDQ_Hyperactivity': -3.088,
    'APQ_P_APQ_P_PP': -1.903,
    'SDQ_SDQ_Externalizing': -1.447,
    'APQ_P_APQ_P_CP': -1.380,
    'APQ_P_APQ_P_INV': -0.437,
    'APQ_P_APQ_P_OPD': -0.676,
    'SDQ_SDQ_Peer_Problems': -0.596
}

# Compute weighted sums
train['female_symptom_score'] = sum(train[feat] * coef for feat, coef in female_coefs.items())
train['male_symptom_score'] = sum(train[feat] * coef for feat, coef in male_coefs.items())

test['female_symptom_score'] = sum(test[feat] * coef for feat, coef in female_coefs.items())
test['male_symptom_score'] = sum(test[feat] * coef for feat, coef in male_coefs.items())

# Preprocessing

### Drop Irrelevant Columns

In [40]:
cols_to_drop = [
    'MRI_Track_Age_at_Scan', 'Basic_Demos_Enroll_Year', 'Basic_Demos_Study_Site',
    'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race',
    'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu',
    'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu',
    'Barratt_Barratt_P2_Occ', 'EHQ_EHQ_Total'
]

train_ids = train['participant_id']
test_ids = test['participant_id'] # I will store them for later usage in grouping in validation why?  I don't want the same user to appear in both train and test. 


for df in (train, test):
    df.set_index('participant_id', inplace=True)
    df.drop(columns=cols_to_drop, errors='ignore', inplace=True)

# for df in (train,test):
#     # df.drop(columns=['participant_id'], inplace=True) 
#     df.set_index('participant_id', inplace=True)
#     df.drop(columns=['EHQ_EHQ_Total'], inplace=True) 
#     df.drop(columns=['MRI_Track_Age_at_Scan'], inplace=True) 

### Setup

In [41]:
# Highlighting important variables. 
# note that I did't deal with the quantative data trq as categorical
# I will use the OneHotEncoder for the categorical data as we have some data trap that I don't think we can use the label encoder for. 

# for feature in trc.columns:
#     train[feature] = train[feature].astype(object)
num_feats = [feature for feature in train.columns if train[feature].dtype == 'float64']
cat_feats = [feature for feature in train.columns if train[feature].dtype == 'object'] # seperate categorical and numerical features help me reteriving them later easily for preprocessing.
target_cols = ['ADHD_Outcome', 'Sex_F']

### Log Transformation

In [42]:
# Features with positive skewness and non-negative values: apply log transformation
# num_feats[:18] Don't transform fMRI data because it has negative values
log_features = [f for f in num_feats[:18] if (train[f] >= 0).all() and scipy.stats.skew(train[f]) > 0.5]  # Only apply to features with significant positive skew

# Apply log transformation for the selected features
for feature in log_features:
    train[feature] = np.log1p(train[feature])  # Apply log(x+1) to handle skewed features
    test[feature] = np.log1p(test[feature])    # Apply log(x+1) to test data as well


### Missing Values

In [43]:
# Find columns with missing values only
train_missing_features_to_impute = train.columns[train.isnull().any()].tolist() # List of features with missing values in train, only 25 and no missing data in fMRI data
test_missing_features_to_impute = test.columns[test.isnull().any()].tolist() # List of features with missing values in test, only 23 and no missing data in fMRI data

# Initialize the imputer
imputer = IterativeImputer(estimator=LassoCV(random_state=42), max_iter=5, random_state=42)

# Impute in-place
train[train_missing_features_to_impute] = imputer.fit_transform(train[train_missing_features_to_impute])
test[test_missing_features_to_impute] = imputer.fit_transform(test[test_missing_features_to_impute])

### Encode Categorical Features 

In [47]:
"""all categorical features have been dropped so no need for this."""

# # Convert all categorical features to strings (to avoid mixed types)
# for feature in cat_feats:
#     train[feature] = train[feature].astype(str)
#     test[feature] = test[feature].astype(str)

# # One-Hot Encoding for categorical features
# encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# for feature in cat_feats:
#     if feature == 'participant_id':  # Skip participant_id since it's not a feature
#         continue

#     # Apply OneHotEncoder
#     train_encoded = encoder.fit_transform(train[[feature]])
#     test_encoded = encoder.transform(test[[feature]])

#     # Convert encoded features to DataFrame and append them to the original data
#     train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out([feature]))
#     test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out([feature]))

#     # Drop the original feature and concatenate the new encoded columns
#     train = pd.concat([train.drop(columns=[feature]), train_encoded_df], axis=1)
#     test = pd.concat([test.drop(columns=[feature]), test_encoded_df], axis=1)

'all categorical features have been dropped so no need for this.'

### Scale Numerical Features

In [48]:
scaler = StandardScaler()

# Only apply scaling to numerical columns that are not part of the target or categorical features
numerical_features = [col for col in train.columns if col not in target_cols and col not in cat_feats]

# Fit scaler on the numerical features of the train set and transform train and test sets
train[numerical_features] = scaler.fit_transform(train[numerical_features])  # Fit and transform for train set
test[numerical_features] = scaler.transform(test[numerical_features])        # Only transform for test set (avoid data leakage)

In [49]:
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

print(non_connectome_features)
print(len(non_connectome_features))

['ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'female_symptom_score', 'male_symptom_score']
18


# Training and Validation

### All data (fMRI and symtpoms) used

In [50]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# Constants
SEED = 7
N_SPLITS = 5
ADHD_WEIGHT = 2  # 2x weight for female ADHD cases

non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models with competition-optimized parameters
model_sex = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Configure cross-validation
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Store competition metrics
adhd_f1_scores = []
sex_f1_scores = []

# Define important features for interaction (update with your actual features)
interaction_features = [    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
                        "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score']  # Use actual column names

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['ADHD_Outcome']), 1):
    print(f"\n=== Fold {fold} ===")
    
    # Split data
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    
    # Extract targets
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Calculate competition weights
    train_weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) * (ADHD_WEIGHT - 1) + 1
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) * (ADHD_WEIGHT - 1) + 1

    # --- Sex Prediction ---
    # Train sex model
    model_sex.fit(X_train.drop(columns=['Sex_F', 'ADHD_Outcome']), y_train_sex, sample_weight=train_weights)
    
    # Get sex probabilities
    sex_train_proba = model_sex.predict_proba(X_train.drop(columns=['Sex_F', 'ADHD_Outcome']))[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val.drop(columns=['Sex_F', 'ADHD_Outcome']))[:, 1]

    # --- ADHD Prediction with Interaction Features ---
    # Create enhanced features
    X_train_adhd = X_train.drop(columns=['Sex_F', 'ADHD_Outcome']).copy()
    X_val_adhd = X_val.drop(columns=['Sex_F', 'ADHD_Outcome']).copy()
    
    # Add sex probability feature
    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba
    
    # Add interaction terms
    for col in interaction_features:
        X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
        X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    # Train ADHD model
    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)
    
    # --- Competition Validation ---
    sex_pred = model_sex.predict(X_val.drop(columns=['Sex_F', 'ADHD_Outcome']))
    adhd_pred = model_adhd.predict(X_val_adhd)
    
    # Calculate weighted F1 scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# --- Final Competition Scoring ---
print("\n=== Competition Results ===")
print(f"Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# Calculate final leaderboard score (average of both F1 scores)
final_score = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
print(f"\nCompetition Score (Normal Mean): {final_score:.4f}")

print(f"Mean Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Mean ADHD_Outcome F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# Calculate final leaderboard score (harmonic mean of average F1s)
final_score = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
print(f"\nCompetition Score (Harmonic Mean): {final_score:.4f}")


=== Fold 1 ===
Sex_F F1: 0.7391
ADHD_Outcome F1: 0.4228

=== Fold 2 ===
Sex_F F1: 0.7345
ADHD_Outcome F1: 0.4407

=== Fold 3 ===
Sex_F F1: 0.6424
ADHD_Outcome F1: 0.4536

=== Fold 4 ===
Sex_F F1: 0.6984
ADHD_Outcome F1: 0.4429

=== Fold 5 ===
Sex_F F1: 0.7077
ADHD_Outcome F1: 0.4737

=== Competition Results ===
Sex_F F1: 0.7044 ± 0.0346
ADHD F1: 0.4467 ± 0.0167

Competition Score (Normal Mean): 0.5756
Mean Sex_F F1: 0.7044 ± 0.0346
Mean ADHD_Outcome F1: 0.4467 ± 0.0167

Competition Score (Harmonic Mean): 0.5467


### All data (fMRI and symtpoms) used, added more female symptoms features to interaction_features

In [52]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# Constants
SEED = 7
N_SPLITS = 5

non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models with competition-optimized parameters
model_sex = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Configure cross-validation
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Store competition metrics
adhd_f1_scores = []
sex_f1_scores = []

# Define important features for interaction (update with your actual features)
interaction_features = [ "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
                        "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score', 
                        'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems', 
                        'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'] 

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['ADHD_Outcome']), 1):
    print(f"\n=== Fold {fold} ===")
    
    # Split data
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    
    # Extract targets
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Calculate competition weights
    train_weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) + 1
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

    # --- Sex Prediction ---
    # Train sex model
    model_sex.fit(X_train.drop(columns=['Sex_F', 'ADHD_Outcome']), y_train_sex, sample_weight=train_weights)
    
    # Get sex probabilities
    sex_train_proba = model_sex.predict_proba(X_train.drop(columns=['Sex_F', 'ADHD_Outcome']))[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val.drop(columns=['Sex_F', 'ADHD_Outcome']))[:, 1]

    # --- ADHD Prediction with Interaction Features ---
    # Create enhanced features
    X_train_adhd = X_train.drop(columns=['Sex_F', 'ADHD_Outcome']).copy()
    X_val_adhd = X_val.drop(columns=['Sex_F', 'ADHD_Outcome']).copy()
    
    # Add sex probability feature
    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba
    
    # Add interaction terms
    for col in interaction_features:
        X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
        X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    # Train ADHD model
    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)
    
    # --- Competition Validation ---
    sex_pred = model_sex.predict(X_val.drop(columns=['Sex_F', 'ADHD_Outcome']))
    adhd_pred = model_adhd.predict(X_val_adhd)
    
    # Calculate weighted F1 scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# --- Final Competition Scoring ---
print("\n=== Competition Results ===")
print(f"Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# Calculate final leaderboard score (average of both F1 scores)
final_score = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
print(f"\nCompetition Score (Normal Mean): {final_score:.4f}")

print(f"Mean Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Mean ADHD_Outcome F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# Calculate final leaderboard score (harmonic mean of average F1s)
final_score = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
print(f"\nCompetition Score (Harmonic Mean): {final_score:.4f}")


=== Fold 1 ===
Sex_F F1: 0.7391
ADHD_Outcome F1: 0.4228

=== Fold 2 ===
Sex_F F1: 0.7345
ADHD_Outcome F1: 0.4512

=== Fold 3 ===
Sex_F F1: 0.6424
ADHD_Outcome F1: 0.4642

=== Fold 4 ===


KeyboardInterrupt: 

### Symptoms only used (no fMRI), added more female symptoms features to interaction_features same as above

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# Constants
SEED = 7
N_SPLITS = 5

# Restrict to non-connectome features
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models
model_sex = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Cross-validation setup
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Score containers
adhd_f1_scores = []
sex_f1_scores = []

# Features used in interaction
interaction_features = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
    "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score', 
    'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems', 
    'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'
] 

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['ADHD_Outcome']), 1):
    print(f"\n=== Fold {fold} ===")

    X_train_full, X_val_full = train.iloc[train_idx], train.iloc[val_idx]
    
    # Targets
    y_train_sex = X_train_full['Sex_F']
    y_val_sex = X_val_full['Sex_F']
    y_train_adhd = X_train_full['ADHD_Outcome']
    y_val_adhd = X_val_full['ADHD_Outcome']

    # Sample weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1

    # Restrict features to non-connectome
    X_train = X_train_full[non_connectome_features].copy()
    X_val = X_val_full[non_connectome_features].copy()

    # --- Sex Model ---
    model_sex.fit(X_train, y_train_sex, sample_weight=train_weights)
    sex_train_proba = model_sex.predict_proba(X_train)[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val)[:, 1]

    # --- ADHD Model ---
    X_train_adhd = X_train.copy()
    X_val_adhd = X_val.copy()

    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba

    # Add interaction features
    for col in interaction_features:
        if col in X_train_adhd.columns:
            X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
            X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)

    # Predict
    sex_pred = model_sex.predict(X_val)
    adhd_pred = model_adhd.predict(X_val_adhd)

    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Final results
print("\n=== Competition Results ===")
print(f"Average Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Average ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# NEW: Calculate harmonic mean per fold first
per_fold_hmeans = [hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)]
final_score_hmean_foldwise = np.mean(per_fold_hmeans)

# Original approach for comparison
final_score_mean = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
final_score_hmean = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])

print(f"\nCompetition Score (Normal Mean): {final_score_mean:.4f}")
print(f"Competition Score (Harmonic Mean of Averages): {final_score_hmean:.4f}")
print(f"Competition Score (Average of Harmonic Means): {final_score_hmean_foldwise:.4f}")  # This is likely what leaderboard uses



=== Fold 1 ===
Sex_F F1: 0.4407
ADHD_Outcome F1: 0.8894

=== Fold 2 ===
Sex_F F1: 0.4978
ADHD_Outcome F1: 0.8856

=== Fold 3 ===
Sex_F F1: 0.5415
ADHD_Outcome F1: 0.8486

=== Fold 4 ===
Sex_F F1: 0.5250
ADHD_Outcome F1: 0.8700

=== Fold 5 ===
Sex_F F1: 0.5804
ADHD_Outcome F1: 0.8874

=== Competition Results ===
Average Sex_F F1: 0.5171 ± 0.0466
Average ADHD F1: 0.8762 ± 0.0154

Competition Score (Normal Mean): 0.6966
Competition Score (Harmonic Mean of Averages): 0.6503
Competition Score (Average of Harmonic Means): 0.6489


### Stratify based on Sex_F

In [63]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# Constants
SEED = 7
N_SPLITS = 5

# Restrict to non-connectome features
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models
model_sex = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Cross-validation setup
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Score containers
adhd_f1_scores = []
sex_f1_scores = []

# Features used in interaction
interaction_features = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
    "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score', 
    'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems', 
    'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'
] 

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['Sex_F']), 1):
    print(f"\n=== Fold {fold} ===")

    X_train_full, X_val_full = train.iloc[train_idx], train.iloc[val_idx]
    
    # Targets
    y_train_sex = X_train_full['Sex_F']
    y_val_sex = X_val_full['Sex_F']
    y_train_adhd = X_train_full['ADHD_Outcome']
    y_val_adhd = X_val_full['ADHD_Outcome']

    # Sample weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1

    # Restrict features to non-connectome
    X_train = X_train_full[non_connectome_features].copy()
    X_val = X_val_full[non_connectome_features].copy()

    # --- Sex Model ---
    model_sex.fit(X_train, y_train_sex, sample_weight=train_weights)
    sex_train_proba = model_sex.predict_proba(X_train)[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val)[:, 1]

    # --- ADHD Model ---
    X_train_adhd = X_train.copy()
    X_val_adhd = X_val.copy()

    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba

    # Add interaction features
    for col in interaction_features:
        if col in X_train_adhd.columns:
            X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
            X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)

    # Predict
    sex_pred = model_sex.predict(X_val)
    adhd_pred = model_adhd.predict(X_val_adhd)

    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Final results
print("\n=== Competition Results ===")
print(f"Average Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Average ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# NEW: Calculate harmonic mean per fold first
per_fold_hmeans = [hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)]
final_score_hmean_foldwise = np.mean(per_fold_hmeans)

# Original approach for comparison
final_score_mean = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
final_score_hmean = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])

print(f"\nCompetition Score (Normal Mean): {final_score_mean:.4f}")
print(f"Competition Score (Harmonic Mean of Averages): {final_score_hmean:.4f}")
print(f"Competition Score (Average of Harmonic Means): {final_score_hmean_foldwise:.4f}")  # This is likely what leaderboard uses



=== Fold 1 ===
Sex_F F1: 0.5344
ADHD_Outcome F1: 0.8856

=== Fold 2 ===
Sex_F F1: 0.5225
ADHD_Outcome F1: 0.8756

=== Fold 3 ===
Sex_F F1: 0.5316
ADHD_Outcome F1: 0.8700

=== Fold 4 ===
Sex_F F1: 0.5103
ADHD_Outcome F1: 0.8973

=== Fold 5 ===
Sex_F F1: 0.5299
ADHD_Outcome F1: 0.8889

=== Competition Results ===
Average Sex_F F1: 0.5258 ± 0.0087
Average ADHD F1: 0.8835 ± 0.0097

Competition Score (Normal Mean): 0.7046
Competition Score (Harmonic Mean of Averages): 0.6592
Competition Score (Average of Harmonic Means): 0.6591


### C=10 for Sex, Stratify on Sex, No fMRI

In [66]:
# Constants
SEED = 7
N_SPLITS = 5
target_cols = ['ADHD_Outcome', 'Sex_F']  # Define target columns

# Define non-connectome features
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models
model_sex = LogisticRegression(
    penalty='l2',
    C=10,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Cross-validation setup
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Score containers
F1s = []
stds = []

for fold_number, (train_index, test_index) in enumerate(cv.split(train[non_connectome_features], train['Sex_F']), 1):
    print(f"\n==== Fold {fold_number} ===")
    
    train_v = train.iloc[train_index]
    test_v = train.iloc[test_index]
    
    # Sample weights
    weights = ((train_v['Sex_F'] == 1) & (train_v['ADHD_Outcome'] == 1)).astype(int) + 1

    # --- Sex Model ---
    model_sex.fit(
        train_v[non_connectome_features], 
        train_v['Sex_F'],
        sample_weight=weights
    )
    sex_pred = model_sex.predict(test_v[non_connectome_features])

    # --- ADHD Model ---
    model_adhd.fit(
        train_v[non_connectome_features],
        train_v['ADHD_Outcome'],
        sample_weight=weights
    )
    adhd_pred = model_adhd.predict(test_v[non_connectome_features])

    # Validation
    valid_idx = test_v[target_cols].notna().all(axis=1)
    valid_testset = test_v.loc[valid_idx, target_cols]

    sex_f1 = f1_score(valid_testset['Sex_F'], sex_pred[valid_idx])
    adhd_f1 = f1_score(valid_testset['ADHD_Outcome'], adhd_pred[valid_idx])

    print(f"Sex_F -> F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome -> F1: {adhd_f1:.4f}")

    F1s.append([adhd_f1, sex_f1])
    stds.append(test_v[target_cols].std())

# Final scoring
F1s = np.array(F1s)
mean_f1_scores = F1s.mean(axis=0)
print("\n==== Overall Results ===")
print("Mean F1-scores:")
for i, target in enumerate(target_cols):
    print(f"    {target} -> F1: {mean_f1_scores[i]:.4f}")

f1_stds = F1s.std(axis=0)
print("\nF1-score stds:")
for i, target in enumerate(target_cols):
    print(f"    {target} -> Std: {f1_stds[i]:.4f}")

print("\nStandard deviations of test sets:")
for std in stds:
    print(f"    {std.to_dict()}")

# Final harmonic mean score
print("\nscore mean (ADHD Sex): ", np.mean(F1s, axis=0))
score = hmean(F1s, axis=0)  # Harmonic mean per fold
score = hmean(score)        # Then overall harmonic mean
print("\nFinal Overall Harmonic Mean (Mimic Leaderboard): ", score)



==== Fold 1 ===
Sex_F -> F1: 0.4910
ADHD_Outcome -> F1: 0.8681

==== Fold 2 ===
Sex_F -> F1: 0.4819
ADHD_Outcome -> F1: 0.8475

==== Fold 3 ===
Sex_F -> F1: 0.4734
ADHD_Outcome -> F1: 0.8523

==== Fold 4 ===
Sex_F -> F1: 0.4767
ADHD_Outcome -> F1: 0.8753

==== Fold 5 ===
Sex_F -> F1: 0.4790
ADHD_Outcome -> F1: 0.8644

==== Overall Results ===
Mean F1-scores:
    ADHD_Outcome -> F1: 0.8615
    Sex_F -> F1: 0.4804

F1-score stds:
    ADHD_Outcome -> Std: 0.0103
    Sex_F -> Std: 0.0060

Standard deviations of test sets:
    {'ADHD_Outcome': 0.46115656157267876, 'Sex_F': 0.4752128960101643}
    {'ADHD_Outcome': 0.47788916775847196, 'Sex_F': 0.4752128960101643}
    {'ADHD_Outcome': 0.4693800748445646, 'Sex_F': 0.47657075200345245}
    {'ADHD_Outcome': 0.44628480198336107, 'Sex_F': 0.475687132835641}
    {'ADHD_Outcome': 0.46833112435802715, 'Sex_F': 0.475687132835641}

score mean (ADHD Sex):  [0.86152011 0.48042091]

Final Overall Harmonic Mean (Mimic Leaderboard):  0.6167636839329982


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# Constants
SEED = 7
N_SPLITS = 5

# Restrict to non-connectome features
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models
model_sex = LogisticRegression(
    penalty='l2',
    C=10,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Cross-validation setup
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Score containers
adhd_f1_scores = []
sex_f1_scores = []

# Features used in interaction
interaction_features = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
    "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score', 
    'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems', 
    'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'
] 

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['Sex_F']), 1):
    print(f"\n=== Fold {fold} ===")

    X_train_full, X_val_full = train.iloc[train_idx], train.iloc[val_idx]
    
    # Targets
    y_train_sex = X_train_full['Sex_F']
    y_val_sex = X_val_full['Sex_F']
    y_train_adhd = X_train_full['ADHD_Outcome']
    y_val_adhd = X_val_full['ADHD_Outcome']

    # Sample weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1

    # Restrict features to non-connectome
    X_train = X_train_full[non_connectome_features].copy()
    X_val = X_val_full[non_connectome_features].copy()

    # --- Sex Model ---
    model_sex.fit(X_train, y_train_sex, sample_weight=train_weights)
    sex_train_proba = model_sex.predict_proba(X_train)[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val)[:, 1]

    # --- ADHD Model ---
    X_train_adhd = X_train.copy()
    X_val_adhd = X_val.copy()

    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba

    # Add interaction features
    for col in interaction_features:
        if col in X_train_adhd.columns:
            X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
            X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)

    # Predict
    sex_pred = model_sex.predict(X_val)
    adhd_pred = model_adhd.predict(X_val_adhd)

    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Final results
print("\n=== Competition Results ===")
print(f"Average Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Average ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# NEW: Calculate harmonic mean per fold first
per_fold_hmeans = [hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)]
final_score_hmean_foldwise = np.mean(per_fold_hmeans)

# Original approach for comparison
final_score_mean = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
final_score_hmean = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])

print(f"\nCompetition Score (Normal Mean): {final_score_mean:.4f}")
print(f"Competition Score (Harmonic Mean of Averages): {final_score_hmean:.4f}")
print(f"Competition Score (Average of Harmonic Means): {final_score_hmean_foldwise:.4f}")  # This is likely what leaderboard uses



=== Fold 1 ===
Sex_F F1: 0.5418
ADHD_Outcome F1: 0.8856

=== Fold 2 ===
Sex_F F1: 0.5286
ADHD_Outcome F1: 0.8756

=== Fold 3 ===
Sex_F F1: 0.5246
ADHD_Outcome F1: 0.8700

=== Fold 4 ===
Sex_F F1: 0.5301
ADHD_Outcome F1: 0.8973

=== Fold 5 ===
Sex_F F1: 0.5272
ADHD_Outcome F1: 0.8889

=== Competition Results ===
Average Sex_F F1: 0.5305 ± 0.0060
Average ADHD F1: 0.8835 ± 0.0097

Competition Score (Normal Mean): 0.7070
Competition Score (Harmonic Mean of Averages): 0.6629
Competition Score (Average of Harmonic Means): 0.6629


In [73]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# ========================
# YOUR APPROACH (ORIGINAL)
# ========================
print("\n" + "="*40)
print("RUNNING YOUR ORIGINAL APPROACH")
print("="*40 + "\n")

# [Rest of your original code up to the final scoring]
# ... [All your original code remains unchanged until the final scoring]

# Final results - Your approach
print("\n=== YOUR FINAL RESULTS ===")
print(f"Average Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Average ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# Your scoring methods
per_fold_hmeans = [hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)]
final_score_hmean_foldwise = np.mean(per_fold_hmeans)
final_score_mean = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
final_score_hmean = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])

print(f"\nYOUR Competition Score (Normal Mean): {final_score_mean:.4f}")
print(f"YOUR Competition Score (Harmonic Mean of Averages): {final_score_hmean:.4f}")
print(f"YOUR Competition Score (Average of Harmonic Means): {final_score_hmean_foldwise:.4f}")

# ========================
# AHMED'S APPROACH 
# ========================
print("\n" + "="*40)
print("RUNNING AHMED'S APPROACH")
print("="*40 + "\n")

# Re-initialize models for Ahmed's approach
model_sex_ahmed = LogisticRegression(
    penalty='l2',
    C=0.02,  # Using Ahmed's C value
    random_state=SEED,
    solver='saga',
    max_iter=1000,
    class_weight='balanced'  # Ahmed's addition
)

model_adhd_ahmed = LogisticRegression(
    penalty='l2',
    C=0.02,  # Using Ahmed's C value
    random_state=SEED,
    solver='saga',
    max_iter=1000,
    class_weight='balanced'  # Ahmed's addition
)

# Ahmed's score containers
F1s = []
stds = []

for fold_number, (train_index, test_index) in enumerate(cv.split(train[non_connectome_features], train['ADHD_Outcome']), 1):
    print(f"\n==== Fold {fold_number} === (Ahmed's Approach)")
    
    train_v = train.iloc[train_index]
    test_v = train.iloc[test_index]
    
    # Ahmed's weight calculation (same as yours)
    weights = ((train_v['Sex_F'] == 1) & (train_v['ADHD_Outcome'] == 1)).astype(int) + 1

    # --- Sex Model ---
    model_sex_ahmed.fit(
        train_v[non_connectome_features], 
        train_v['Sex_F'],
        sample_weight=weights
    )
    sex_pred = model_sex_ahmed.predict(test_v[non_connectome_features])

    # --- ADHD Model ---
    model_adhd_ahmed.fit(
        train_v[non_connectome_features],
        train_v['ADHD_Outcome'],
        sample_weight=weights
    )
    adhd_pred = model_adhd_ahmed.predict(test_v[non_connectome_features])

    # Ahmed's validation with null check
    valid_idx = test_v[target_cols].notna().all(axis=1)
    valid_testset = test_v.loc[valid_idx, target_cols]

    sex_f1 = f1_score(valid_testset['Sex_F'], sex_pred[valid_idx])
    adhd_f1 = f1_score(valid_testset['ADHD_Outcome'], adhd_pred[valid_idx])

    print(f"Sex_F -> F1: {sex_f1:.4f} (Ahmed)")
    print(f"ADHD_Outcome -> F1: {adhd_f1:.4f} (Ahmed)")

    F1s.append([adhd_f1, sex_f1])
    stds.append(test_v[target_cols].std())

# Ahmed's final scoring
F1s = np.array(F1s)
mean_f1_scores = F1s.mean(axis=0)
print("\n==== AHMED'S OVERALL RESULTS ====")
print("Mean F1-scores:")
for i, target in enumerate(target_cols):
    print(f"    {target} -> F1: {mean_f1_scores[i]:.4f}")

# ========================
# YOUR APPROACH 
# ========================
print("\n=== RUNNING YOUR APPROACH ===\n")

# [Your original code execution...]

your_results = {
    "Sex_F F1 (mean)": np.mean(sex_f1_scores),
    "ADHD F1 (mean)": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Harmonic Mean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Average of Harmonic Means": np.mean([hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)])
}

# ========================
# AHMED'S APPROACH 
# ========================
print("\n=== RUNNING AHMED'S APPROACH ===\n")

# [Ahmed's code execution...]

ahmed_results = {
    "Sex_F F1 (mean)": F1s[:,1].mean(),
    "ADHD F1 (mean)": F1s[:,0].mean(),
    "Normal Mean": np.mean(F1s.mean(axis=0)),
    "Harmonic Mean of Averages": hmean(F1s.mean(axis=0)),
    "Average of Harmonic Means": np.mean([hmean(fold) for fold in F1s]),
    "Double Harmonic Mean": hmean(hmean(F1s, axis=0))
}
# ========================
# SIDE-BY-SIDE COMPARISON
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<30} | {:>15} | {:>15}".format("Metric", "Your Approach", "Ahmed's Approach"))
print("-" * 70)
for key in your_results:
    if key in ahmed_results:
        print("{:<30} | {:>15.4f} | {:>15.4f}".format(key, your_results[key], ahmed_results[key]))
# Show Ahmed's special score separately



RUNNING YOUR ORIGINAL APPROACH


=== YOUR FINAL RESULTS ===
Average Sex_F F1: 0.7010 ± 0.0189
Average ADHD F1: 0.4598 ± 0.0263

YOUR Competition Score (Normal Mean): 0.5804
YOUR Competition Score (Harmonic Mean of Averages): 0.5554
YOUR Competition Score (Average of Harmonic Means): 0.5545

RUNNING AHMED'S APPROACH


==== Fold 1 === (Ahmed's Approach)
Sex_F -> F1: 0.5467 (Ahmed)
ADHD_Outcome -> F1: 0.8250 (Ahmed)

==== Fold 2 === (Ahmed's Approach)
Sex_F -> F1: 0.5782 (Ahmed)
ADHD_Outcome -> F1: 0.8418 (Ahmed)

==== Fold 3 === (Ahmed's Approach)
Sex_F -> F1: 0.4552 (Ahmed)
ADHD_Outcome -> F1: 0.7925 (Ahmed)

==== Fold 4 === (Ahmed's Approach)
Sex_F -> F1: 0.5072 (Ahmed)
ADHD_Outcome -> F1: 0.8483 (Ahmed)

==== Fold 5 === (Ahmed's Approach)
Sex_F -> F1: 0.5017 (Ahmed)
ADHD_Outcome -> F1: 0.8344 (Ahmed)

==== AHMED'S OVERALL RESULTS ====
Mean F1-scores:
    ADHD_Outcome -> F1: 0.8284
    Sex_F -> F1: 0.5178

=== RUNNING YOUR APPROACH ===


=== RUNNING AHMED'S APPROACH ===


=== FINAL CO

In [ ]:

# ========================
# YOUR APPROACH 
# ========================
print("\n=== RUNNING YOUR APPROACH ===\n")

# [Your original code execution...]

your_results = {
    "Sex_F F1 (mean)": np.mean(sex_f1_scores),
    "ADHD F1 (mean)": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Harmonic Mean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Average of Harmonic Means": np.mean([hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)])
}

# ========================
# AHMED'S APPROACH 
# ========================
print("\n=== RUNNING AHMED'S APPROACH ===\n")

# [Ahmed's code execution...]

ahmed_results = {
    "Sex_F F1 (mean)": F1s[:,1].mean(),
    "ADHD F1 (mean)": F1s[:,0].mean(),
    "Normal Mean": np.mean(F1s.mean(axis=0)),
    "Harmonic Mean of Averages": hmean(F1s.mean(axis=0)),
    "Average of Harmonic Means": np.mean([hmean(fold) for fold in F1s]),
    "Double Harmonic Mean": hmean(hmean(F1s, axis=0))
}
# ========================
# SIDE-BY-SIDE COMPARISON
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<30} | {:>15} | {:>15}".format("Metric", "Your Approach", "Ahmed's Approach"))
print("-" * 70)
for key in your_results:
    if key in ahmed_results:
        print("{:<30} | {:>15.4f} | {:>15.4f}".format(key, your_results[key], ahmed_results[key]))
# Show Ahmed's special score separately



=== RUNNING YOUR APPROACH ===


=== RUNNING AHMED'S APPROACH ===


=== FINAL COMPARISON ===
Metric                         |   Your Approach | Ahmed's Approach
----------------------------------------------------------------------
Sex_F F1 (mean)                |          0.7010 |          0.5178
ADHD F1 (mean)                 |          0.4598 |          0.8284
Normal Mean                    |          0.5804 |          0.6731
Harmonic Mean of Averages      |          0.5554 |          0.6373
Average of Harmonic Means      |          0.5545 |          0.6366


### train ADHD model on symptoms only, sex model on all data or fMRI only

In [81]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# Constants
SEED = 7
N_SPLITS = 5
target_cols = ['ADHD_Outcome', 'Sex_F']

# Feature selection
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]
# ========================
# YOUR APPROACH (ADJUSTED)
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

# [Previous initialization code remains the same...]

# Key fix: Use the same features for stratification as for training
full_features = [col for col in train.columns if col not in target_cols]

# Cross-validation - stratify on the full features
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores = []
sex_f1_scores = []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    print(f"\n=== Fold {fold} (Adjusted) ===")
    
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1
    
    # --- Sex model: use ALL features ---
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=train_weights)
    sex_pred = model_sex.predict(X_val[full_features])
    
    # --- ADHD model: use only non-connectome features ---
    model_adhd.fit(X_train[non_connectome_features], y_train_adhd, sample_weight=train_weights)
    adhd_pred = model_adhd.predict(X_val[non_connectome_features])
    
    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Your adjusted results
your_adjusted_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "HMean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Avg of HMeans": np.mean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)])
}

# [Rest of the code remains the same...]




# ========================
# AHMED'S APPROACH (ADJUSTED)
# ========================
print("\n=== AHMED'S APPROACH (ADJUSTED) ===\n")

# Initialize Ahmed's models with your requested changes
model_sex_ahmed = LogisticRegression(
    penalty='l2', 
    C=10,  # Changed from 0.02 to 10 for sex model
    random_state=SEED,
    solver='saga', 
    max_iter=1000,
    # Removed class_weight='balanced'
)

model_adhd_ahmed = LogisticRegression(
    penalty='l2', 
    C=0.02,  # Kept original C value for ADHD
    random_state=SEED,
    solver='saga', 
    max_iter=1000,
    # Removed class_weight='balanced'
)

# Cross-validation - now stratifying by Sex_F
F1s = []
for fold, (train_idx, val_idx) in enumerate(cv.split(train[non_connectome_features], train['Sex_F']), 1):
    print(f"\n=== Fold {fold} (Ahmed Adjusted) ===")
    
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) + 1
    
    # --- Sex model: use full feature set ---
    full_features = [col for col in train.columns if col not in target_cols]
    model_sex_ahmed.fit(X_train[full_features], X_train['Sex_F'], sample_weight=weights)
    sex_pred = model_sex_ahmed.predict(X_val[full_features])
    
    # --- ADHD model: use non-connectome features ---
    model_adhd_ahmed.fit(X_train[non_connectome_features], X_train['ADHD_Outcome'], sample_weight=weights)
    adhd_pred = model_adhd_ahmed.predict(X_val[non_connectome_features])
    
    # Validation
    valid_idx = X_val[target_cols].notna().all(axis=1)
    sex_f1 = f1_score(X_val.loc[valid_idx, 'Sex_F'], sex_pred[valid_idx])
    adhd_f1 = f1_score(X_val.loc[valid_idx, 'ADHD_Outcome'], adhd_pred[valid_idx])
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    F1s.append([adhd_f1, sex_f1])

F1s = np.array(F1s)

# Adjusted results without Double Harmonic Mean
ahmed_adjusted_results = {
    "Sex_F F1": F1s[:,1].mean(),
    "ADHD F1": F1s[:,0].mean(),
    "Normal Mean": np.mean(F1s.mean(axis=0)),
    "HMean of Averages": hmean(F1s.mean(axis=0)),
    "Avg of HMeans": np.mean([hmean(fold) for fold in F1s])
    # Removed "Double HMean"
}

# ========================
# UPDATED COMPARISON
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<30} | {:>15} | {:>15}".format("Metric", "Your Approach", "Ahmed's Adjusted"))
print("-" * 70)
for key in your_results:
    if key in ahmed_adjusted_results:
        print("{:<30} | {:>15.4f} | {:>15.4f}".format(key, your_results[key], ahmed_adjusted_results[key]))


=== YOUR APPROACH (ADJUSTED) ===


=== Fold 1 (Adjusted) ===
Sex_F F1: 0.7221
ADHD_Outcome F1: 0.8856

=== Fold 2 (Adjusted) ===
Sex_F F1: 0.6894
ADHD_Outcome F1: 0.8736

=== Fold 3 (Adjusted) ===
Sex_F F1: 0.7251
ADHD_Outcome F1: 0.8680

=== Fold 4 (Adjusted) ===
Sex_F F1: 0.6783
ADHD_Outcome F1: 0.8977

=== Fold 5 (Adjusted) ===
Sex_F F1: 0.6901
ADHD_Outcome F1: 0.8889

=== AHMED'S APPROACH (ADJUSTED) ===


=== Fold 1 (Ahmed Adjusted) ===
Sex_F F1: 0.6167
ADHD_Outcome F1: 0.8681

=== Fold 2 (Ahmed Adjusted) ===
Sex_F F1: 0.6066
ADHD_Outcome F1: 0.8475

=== Fold 3 (Ahmed Adjusted) ===
Sex_F F1: 0.6441
ADHD_Outcome F1: 0.8523

=== Fold 4 (Ahmed Adjusted) ===
Sex_F F1: 0.5749
ADHD_Outcome F1: 0.8753

=== Fold 5 (Ahmed Adjusted) ===
Sex_F F1: 0.5887
ADHD_Outcome F1: 0.8644

=== FINAL COMPARISON ===
Metric                         |   Your Approach | Ahmed's Adjusted
----------------------------------------------------------------------
Sex_F F1                       |          0.5305 |  

In [84]:
# Adding double hmean
# ========================
# YOUR APPROACH (ADJUSTED)
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

# [Previous code remains identical until the results section...]

# Your adjusted results - now with teammate's scoring method
your_adjusted_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "HMean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Avg of HMeans": np.mean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)]),
    # Teammate's scoring method added:
    "Double HMean": hmean(hmean(np.column_stack([sex_f1_scores, adhd_f1_scores]), axis=0))
}

# ========================
# AHMED'S APPROACH (ADJUSTED)
# ========================
print("\n=== AHMED'S APPROACH (ADJUSTED) ===\n")

# [Previous code remains identical until the results section...]

# Ahmed's adjusted results - now with teammate's scoring method
ahmed_adjusted_results = {
    "Sex_F F1": F1s[:,1].mean(),
    "ADHD F1": F1s[:,0].mean(),
    "Normal Mean": np.mean(F1s.mean(axis=0)),
    "HMean of Averages": hmean(F1s.mean(axis=0)),
    "Avg of HMeans": np.mean([hmean(fold) for fold in F1s]),
    # Teammate's scoring method added:
    "Double HMean": hmean(hmean(F1s, axis=0))
}

# ========================
# UPDATED COMPARISON (WITH BETTER ORDERING)
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<30} | {:>15} | {:>15}".format("Metric", "Your Approach", "Ahmed's Adjusted"))
print("-" * 70)

# Define the preferred order of metrics
preferred_order = [
    "Sex_F F1",
    "ADHD F1",
    "Normal Mean", 
    "HMean of Averages",
    "Avg of HMeans",
    "Double HMean"
]

# Print metrics in the specified order
for key in preferred_order:
    your_val = your_adjusted_results.get(key, "-")
    ahmed_val = ahmed_adjusted_results.get(key, "-")
    
    if isinstance(your_val, float):
        your_val = f"{your_val:.4f}"
    if isinstance(ahmed_val, float):
        ahmed_val = f"{ahmed_val:.4f}"
    
    print("{:<30} | {:>15} | {:>15}".format(key, your_val, ahmed_val))


=== YOUR APPROACH (ADJUSTED) ===


=== AHMED'S APPROACH (ADJUSTED) ===


=== FINAL COMPARISON ===
Metric                         |   Your Approach | Ahmed's Adjusted
----------------------------------------------------------------------
Sex_F F1                       |          0.7010 |          0.6062
ADHD F1                        |          0.8828 |          0.8615
Normal Mean                    |          0.7919 |          0.7339
HMean of Averages              |          0.7814 |          0.7116
Avg of HMeans                  |          0.7812 |          0.7112
Double HMean                   |          0.7811 |          0.7110


In [ ]:
# Constants
SEED = 7
N_SPLITS = 5
target_cols = ['ADHD_Outcome', 'Sex_F']

# Feature selection
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]
# ========================
# YOUR APPROACH (ADJUSTED)
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

# [Previous initialization code remains the same...]

# Key fix: Use the same features for stratification as for training
full_features = [col for col in train.columns if col not in target_cols]

# Cross-validation - stratify on the full features
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores = []
sex_f1_scores = []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    print(f"\n=== Fold {fold} (Adjusted) ===")
    
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1
    
    # --- Sex model: use ALL features ---
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=train_weights)
    sex_pred = model_sex.predict(X_val[full_features])
    
    # --- ADHD model: use only non-connectome features ---
    model_adhd.fit(X_train[non_connectome_features], y_train_adhd, sample_weight=train_weights)
    adhd_pred = model_adhd.predict(X_val[non_connectome_features])
    
    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Your adjusted results
your_adjusted_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "HMean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Avg of HMeans": np.mean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)]),
    # Teammate's scoring method added:
    "Double HMean": hmean(hmean(np.column_stack([sex_f1_scores, adhd_f1_scores]), axis=0))
}






# ========================
# AHMED'S APPROACH (ADJUSTED)
# ========================
print("\n=== AHMED'S APPROACH (ADJUSTED) ===\n")

# Initialize Ahmed's models with your requested changes
model_sex_ahmed = LogisticRegression(
    penalty='l2', 
    C=10,  # Changed from 0.02 to 10 for sex model
    random_state=SEED,
    solver='saga', 
    max_iter=1000,
    # Removed class_weight='balanced'
)

model_adhd_ahmed = LogisticRegression(
    penalty='l2', 
    C=0.02,  # Kept original C value for ADHD
    random_state=SEED,
    solver='saga', 
    max_iter=1000,
    # Removed class_weight='balanced'
)

# Cross-validation - now stratifying by Sex_F
F1s = []
for fold, (train_idx, val_idx) in enumerate(cv.split(train[non_connectome_features], train['Sex_F']), 1):
    print(f"\n=== Fold {fold} (Ahmed Adjusted) ===")
    
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) + 1
    
    # --- Sex model: use full feature set ---
    full_features = [col for col in train.columns if col not in target_cols]
    model_sex_ahmed.fit(X_train[full_features], X_train['Sex_F'], sample_weight=weights)
    sex_pred = model_sex_ahmed.predict(X_val[full_features])
    
    # --- ADHD model: use non-connectome features ---
    model_adhd_ahmed.fit(X_train[non_connectome_features], X_train['ADHD_Outcome'], sample_weight=weights)
    adhd_pred = model_adhd_ahmed.predict(X_val[non_connectome_features])
    
    # Validation
    valid_idx = X_val[target_cols].notna().all(axis=1)
    sex_f1 = f1_score(X_val.loc[valid_idx, 'Sex_F'], sex_pred[valid_idx])
    adhd_f1 = f1_score(X_val.loc[valid_idx, 'ADHD_Outcome'], adhd_pred[valid_idx])
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    F1s.append([adhd_f1, sex_f1])

F1s = np.array(F1s)

# Adjusted results without Double Harmonic Mean
ahmed_adjusted_results = {
    "Sex_F F1": F1s[:,1].mean(),
    "ADHD F1": F1s[:,0].mean(),
    "Normal Mean": np.mean(F1s.mean(axis=0)),
    "HMean of Averages": hmean(F1s.mean(axis=0)),
    "Avg of HMeans": np.mean([hmean(fold) for fold in F1s]),
    # Teammate's scoring method added:
    "Double HMean": hmean(hmean(F1s, axis=0))
}

# ========================
# UPDATED COMPARISON (WITH BETTER ORDERING)
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<30} | {:>15} | {:>15}".format("Metric", "Your Approach", "Ahmed's Adjusted"))
print("-" * 70)

# Define the preferred order of metrics
preferred_order = [
    "Sex_F F1",
    "ADHD F1",
    "Normal Mean", 
    "HMean of Averages",
    "Avg of HMeans",
    "Double HMean"
]

# Print metrics in the specified order
for key in preferred_order:
    your_val = your_adjusted_results.get(key, "-")
    ahmed_val = ahmed_adjusted_results.get(key, "-")
    
    if isinstance(your_val, float):
        your_val = f"{your_val:.4f}"
    if isinstance(ahmed_val, float):
        ahmed_val = f"{ahmed_val:.4f}"
    
    print("{:<30} | {:>15} | {:>15}".format(key, your_val, ahmed_val))

In [86]:
# ========================
# YOUR APPROACH (ADJUSTED)
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

# Initialize models
model_sex = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
model_adhd = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# Cross-validation - stratify using ALL features
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores, sex_f1_scores = [], []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Weights
    weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    
    # --- Sex model (all features) ---
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=weights)
    sex_pred = model_sex.predict(X_val[full_features])
    
    # --- ADHD model (non-connectome only) ---
    model_adhd.fit(X_train[non_connectome_features], y_train_adhd, sample_weight=weights)
    adhd_pred = model_adhd.predict(X_val[non_connectome_features])

    # Correct sample weights for validation set
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Results
your_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Double HMean": hmean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)])
}

# ========================
# AHMED'S APPROACH (FIXED)
# ========================
print("\n=== AHMED'S APPROACH (FIXED) ===\n")

# Initialize models (same as yours)
model_sex_ahmed = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
model_adhd_ahmed = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# Cross-validation - identical splitting
F1s = []
for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) + 1
    
    # --- Sex model (all features) ---
    model_sex_ahmed.fit(X_train[full_features], X_train['Sex_F'], sample_weight=weights)
    sex_pred = model_sex_ahmed.predict(X_val[full_features])
    
    # --- ADHD model (non-connectome only) ---
    model_adhd_ahmed.fit(X_train[non_connectome_features], X_train['ADHD_Outcome'], sample_weight=weights)
    adhd_pred = model_adhd_ahmed.predict(X_val[non_connectome_features])
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

    # Scores
    F1s.append([
        f1_score(X_val['ADHD_Outcome'], adhd_pred, sample_weight=val_weights),
        f1_score(X_val['Sex_F'], sex_pred, sample_weight=val_weights)
    ])
    

F1s = np.array(F1s)
ahmed_results = {
    "Sex_F F1": F1s[:,1].mean(),
    "ADHD F1": F1s[:,0].mean(),
    "Double HMean": hmean(hmean(F1s, axis=0))
}

# ========================
# FINAL COMPARISON
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<15} | {:>12} | {:>12}".format("Metric", "Your", "Ahmed's"))
print("-" * 45)
for metric in ["Sex_F F1", "ADHD F1", "Double HMean"]:
    print("{:<15} | {:>12.4f} | {:>12.4f}".format(
        metric, 
        your_results[metric], 
        ahmed_results[metric]
    ))


=== YOUR APPROACH (ADJUSTED) ===


=== AHMED'S APPROACH (FIXED) ===


=== FINAL COMPARISON ===
Metric          |         Your |      Ahmed's
---------------------------------------------
Sex_F F1        |       0.7010 |       0.7010
ADHD F1         |       0.8828 |       0.8828
Double HMean    |       0.7811 |       0.7811


#### Changing seed to see if CV is overfitting or consistent results

In [ ]:
# Experiment with different seed, default thresholding
# ========================
# YOUR APPROACH (ADJUSTED)
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

SEED = 42
# Initialize models
model_sex = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
model_adhd = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# Cross-validation - stratify using ALL features
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores, sex_f1_scores = [], []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Weights
    weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    
    # --- Sex model (all features) ---
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=weights)
    sex_pred = model_sex.predict(X_val[full_features])
    
    # --- ADHD model (non-connectome only) ---
    model_adhd.fit(X_train[non_connectome_features], y_train_adhd, sample_weight=weights)
    adhd_pred = model_adhd.predict(X_val[non_connectome_features])

    # Correct sample weights for validation set
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Results
your_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Double HMean": hmean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)])
}

# ========================
# AHMED'S APPROACH (FIXED)
# ========================
print("\n=== AHMED'S APPROACH (FIXED) ===\n")

# Initialize models (same as yours)
model_sex_ahmed = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
model_adhd_ahmed = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# Cross-validation - identical splitting
F1s = []
for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) + 1
    
    # --- Sex model (all features) ---
    model_sex_ahmed.fit(X_train[full_features], X_train['Sex_F'], sample_weight=weights)
    sex_pred = model_sex_ahmed.predict(X_val[full_features])
    
    # --- ADHD model (non-connectome only) ---
    model_adhd_ahmed.fit(X_train[non_connectome_features], X_train['ADHD_Outcome'], sample_weight=weights)
    adhd_pred = model_adhd_ahmed.predict(X_val[non_connectome_features])
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

    # Scores
    F1s.append([
        f1_score(X_val['ADHD_Outcome'], adhd_pred, sample_weight=val_weights),
        f1_score(X_val['Sex_F'], sex_pred, sample_weight=val_weights)
    ])
    

F1s = np.array(F1s)
ahmed_results = {
    "Sex_F F1": F1s[:,1].mean(),
    "ADHD F1": F1s[:,0].mean(),
    "Double HMean": hmean(hmean(F1s, axis=0))
}

# ========================
# FINAL COMPARISON
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<15} | {:>12} | {:>12}".format("Metric", "Your", "Ahmed's"))
print("-" * 45)
for metric in ["Sex_F F1", "ADHD F1", "Double HMean"]:
    print("{:<15} | {:>12.4f} | {:>12.4f}".format(
        metric, 
        your_results[metric], 
        ahmed_results[metric]
    ))

"""
Output
Seed: 42 Normal thresholds: 0.5
=== FINAL COMPARISON ===
Metric          |         Your |      Ahmed's
---------------------------------------------
Sex_F F1        |       0.7010 |       0.7010
ADHD F1         |       0.8828 |       0.8828
Double HMean    |       0.7811 |       0.7811
"""


"""
Thresholds: 0.3 sex, 0.4 adhd
=== FINAL COMPARISON ===
Metric          |         Your |      Ahmed's
---------------------------------------------
Sex_F F1        |       0.7088 |       0.7088
ADHD F1         |       0.8801 |       0.8801
Double HMean    |       0.7848 |       0.7848
"""


=== YOUR APPROACH (ADJUSTED) ===


=== AHMED'S APPROACH (FIXED) ===


=== FINAL COMPARISON ===
Metric          |         Your |      Ahmed's
---------------------------------------------
Sex_F F1        |       0.7088 |       0.7088
ADHD F1         |       0.8801 |       0.8801
Double HMean    |       0.7848 |       0.7848


"\n=== FINAL COMPARISON ===\nMetric          |         Your |      Ahmed's\n---------------------------------------------\nSex_F F1        |       0.7010 |       0.7010\nADHD F1         |       0.8828 |       0.8828\nDouble HMean    |       0.7811 |       0.7811\n"

#### Threshold: Sex 0.3 all data, ADHD 0.4 only connectome data

In [ ]:
#Threshold: Sex 0.3 all data, ADHD 0.4 only connectome data
# ========================
# YOUR APPROACH (ADJUSTED) WITH THRESHOLDS
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

SEED = 42
SEX_THRESHOLD = 0.3
ADHD_THRESHOLD = 0.4

# Initialize models
model_sex = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
model_adhd = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# Cross-validation
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores, sex_f1_scores = [], []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    
    # --- Train and predict for Sex
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=weights)
    sex_proba = model_sex.predict_proba(X_val[full_features])[:, 1]
    sex_pred = (sex_proba >= SEX_THRESHOLD).astype(int)

    # --- Train and predict for ADHD
    model_adhd.fit(X_train[non_connectome_features], y_train_adhd, sample_weight=weights)
    adhd_proba = model_adhd.predict_proba(X_val[non_connectome_features])[:, 1]
    adhd_pred = (adhd_proba >= ADHD_THRESHOLD).astype(int)

    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

    # === Fold Output
    print(f"=== Fold {fold} ===")
    print(f"Sex_F -> F1: {sex_f1:.4f}")
    print(f"Outcome ADHD -> F1: {adhd_f1:.4f}\n")


# ========================
# FINAL RESULTS
# ========================
your_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "HMean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Avg of HMeans": np.mean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)]),
    "Double HMean": hmean(hmean(np.column_stack([sex_f1_scores, adhd_f1_scores]), axis=0))
}

# Clean and simplified output
print("=== FINAL COMPARISON ===")
print("{:<30} | {:>15}".format("Metric", "Your Approach"))
print("-" * 50)
for key in [
    "Sex_F F1",
    "ADHD F1",
    "Normal Mean", 
    "HMean of Averages",
    "Avg of HMeans",
    "Double HMean"
]:
    print("{:<30} | {:>15.4f}".format(key, your_results[key]))


"""
Output:
=== YOUR APPROACH (ADJUSTED) ===

=== Fold 1 ===
Sex_F -> F1: 0.6798
Outcome ADHD -> F1: 0.9213

=== Fold 2 ===
Sex_F -> F1: 0.7104
Outcome ADHD -> F1: 0.8547

=== Fold 3 ===
Sex_F -> F1: 0.7088
Outcome ADHD -> F1: 0.8815

=== Fold 4 ===
Sex_F -> F1: 0.6786
Outcome ADHD -> F1: 0.8705

=== Fold 5 ===
Sex_F -> F1: 0.7024
Outcome ADHD -> F1: 0.8973

=== FINAL COMPARISON ===
Metric                         |   Your Approach
--------------------------------------------------
Sex_F F1                       |          0.6960
ADHD F1                        |          0.8850
Normal Mean                    |          0.7905
HMean of Averages              |          0.7792
Avg of HMeans                  |          0.7789
Double HMean                   |          0.7788
"""


=== YOUR APPROACH (ADJUSTED) ===

=== Fold 1 ===
Sex_F -> F1: 0.6798
Outcome ADHD -> F1: 0.9213

=== Fold 2 ===
Sex_F -> F1: 0.7104
Outcome ADHD -> F1: 0.8547

=== Fold 3 ===
Sex_F -> F1: 0.7088
Outcome ADHD -> F1: 0.8815

=== Fold 4 ===
Sex_F -> F1: 0.6786
Outcome ADHD -> F1: 0.8705

=== Fold 5 ===
Sex_F -> F1: 0.7024
Outcome ADHD -> F1: 0.8973

=== FINAL COMPARISON ===
Metric                         |   Your Approach
--------------------------------------------------
Sex_F F1                       |          0.6960
ADHD F1                        |          0.8850
Normal Mean                    |          0.7905
HMean of Averages              |          0.7792
Avg of HMeans                  |          0.7789
Double HMean                   |          0.7788


In [ ]:
# All data for both models

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean
import numpy as np

# ========================
# YOUR APPROACH (ALL FEATURES FOR BOTH MODELS)
# ========================
print("\n=== YOUR APPROACH (ALL FEATURES) ===\n")

SEED = 42
SEX_THRESHOLD = 0.3
ADHD_THRESHOLD = 0.4

# Initialize models
model_sex = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
model_adhd = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# Cross-validation
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores, sex_f1_scores = [], []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    
    # --- Train and predict for Sex (using all features)
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=weights)
    sex_proba = model_sex.predict_proba(X_val[full_features])[:, 1]
    sex_pred = (sex_proba >= SEX_THRESHOLD).astype(int)

    # --- Train and predict for ADHD (now also using all features)
    model_adhd.fit(X_train[full_features], y_train_adhd, sample_weight=weights)
    adhd_proba = model_adhd.predict_proba(X_val[full_features])[:, 1]
    adhd_pred = (adhd_proba >= ADHD_THRESHOLD).astype(int)

    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

    # === Fold Output
    print(f"=== Fold {fold} ===")
    print(f"Sex_F -> F1: {sex_f1:.4f}")
    print(f"Outcome ADHD -> F1: {adhd_f1:.4f}\n")


# ========================
# FINAL RESULTS
# ========================
your_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "HMean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Avg of HMeans": np.mean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)]),
    "Double HMean": hmean(hmean(np.column_stack([sex_f1_scores, adhd_f1_scores]), axis=0))
}

# Clean and simplified output
print("=== FINAL COMPARISON ===")
print("{:<30} | {:>15}".format("Metric", "Your Approach"))
print("-" * 50)
for key in [
    "Sex_F F1",
    "ADHD F1",
    "Normal Mean", 
    "HMean of Averages",
    "Avg of HMeans",
    "Double HMean"
]:
    print("{:<30} | {:>15.4f}".format(key, your_results[key]))


"""
output incomplete becuz of error:
=== YOUR APPROACH (ALL FEATURES) ===

=== Fold 1 ===
Sex_F -> F1: 0.6798
Outcome ADHD -> F1: 0.5130

=== Fold 2 ===
Sex_F -> F1: 0.7104
Outcome ADHD -> F1: 0.3852

=== Fold 3 ===
Sex_F -> F1: 0.7088
Outcome ADHD -> F1: 0.4422

=== Fold 4 ===
Sex_F -> F1: 0.6786
Outcome ADHD -> F1: 0.4859
"""



=== YOUR APPROACH (ALL FEATURES) ===



KeyboardInterrupt: 

### Trying LGBM

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean
import numpy as np

# ========================
# YOUR APPROACH (LIGHTGBM)
# ========================
print("\n=== YOUR APPROACH (LIGHTGBM) ===\n")

SEED = 42
SEX_THRESHOLD = 0.3
ADHD_THRESHOLD = 0.4

# Initialize models
model_sex = LGBMClassifier(random_state=SEED, n_estimators=100, learning_rate=0.05, verbos = -1)
model_adhd = LGBMClassifier(random_state=SEED, n_estimators=100, learning_rate=0.05, verbos = -1)

# Cross-validation
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores, sex_f1_scores = [], []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1

    # --- Train and predict for Sex (all features)
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=weights)
    sex_proba = model_sex.predict_proba(X_val[full_features])[:, 1]
    sex_pred = (sex_proba >= SEX_THRESHOLD).astype(int)

    # --- Train and predict for ADHD (all features)
    model_adhd.fit(X_train[full_features], y_train_adhd, sample_weight=weights)
    adhd_proba = model_adhd.predict_proba(X_val[full_features])[:, 1]
    adhd_pred = (adhd_proba >= ADHD_THRESHOLD).astype(int)

    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

    # === Fold Output
    print(f"=== Fold {fold} ===")
    print(f"Sex_F -> F1: {sex_f1:.4f}")
    print(f"Outcome ADHD -> F1: {adhd_f1:.4f}\n")


# ========================
# FINAL RESULTS
# ========================
your_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "HMean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Avg of HMeans": np.mean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)]),
    "Double HMean": hmean(hmean(np.column_stack([sex_f1_scores, adhd_f1_scores]), axis=0))
}

# Clean and simplified output
print("=== FINAL COMPARISON ===")
print("{:<30} | {:>15}".format("Metric", "Your Approach"))
print("-" * 50)
for key in [
    "Sex_F F1",
    "ADHD F1",
    "Normal Mean", 
    "HMean of Averages",
    "Avg of HMeans",
    "Double HMean"
]:
    print("{:<30} | {:>15.4f}".format(key, your_results[key]))



=== YOUR APPROACH (LIGHTGBM) ===

[LightGBM] [Warning] Unknown parameter: verbos
[LightGBM] [Warning] Unknown parameter: verbos
[LightGBM] [Info] Number of positive: 333, number of negative: 637


# Importing Dependencies

### the notebook in a nutshell 
- importing 
software dependencies we need
- load_data (please read its description it supposed to be very easy to deal with it)

###  preprocesssing 
- I stored some important variables, then I used log transformation for long tailed distributions

- iterative imputer just like what `MAAB` Advised

- transormed the categorical features to be from the string type just to avoid some fitting isssues cause by the 
OneHotEncoder.

- droped the `participant_ID` after storing it for stratification

- I scaled quantative data and also FMRI data 

### modeling
- base model is logisitc regression with l2 regulization (fancy name for punishing the model to stop overfitting to noise) 
- wrapped inside a multioutput classifier that just gives it the ability to predict 2 targets simultaneously
### validation 
- 5 splits stratifiedKFold works fine no need for grouping as participants are unique 
- regular stratified  = 0.5901173137436236
- RepeatedStratifiedKFold =0.5865925294336686 with 7 minutese delay

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score  
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from scipy.stats import hmean
from sklearn.multioutput import MultiOutputClassifier
import scipy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer #it might not work directly if not try the following code line

from sklearn.model_selection import StratifiedGroupKFold,StratifiedKFold,RepeatedStratifiedKFold
from pathlib import Path
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

# Reading Data

In [ ]:
"""
just drop the path of your new data directory nothing more than that. 

"""
path = r"C:\Users\Maab\Desktop\ADHD_Kaggle_Competition\Repo\WiDS-Datathon-2025\Data\raw"

def read_data(base_path:str) -> pd.DataFrame :
    path = Path(base_path)
    trc=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_CATEGORICAL_METADATA_new.xlsx')
    trq=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_QUANTITATIVE_METADATA_new.xlsx')
    trf=pd.read_csv(path   /'TRAIN_NEW'  / 'TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
    trs=pd.read_excel(path /'TRAIN_NEW'  / 'TRAINING_SOLUTIONS.xlsx')  
    tsc=pd.read_excel(path /'TEST'      / 'TEST_CATEGORICAL.xlsx')
    tsq=pd.read_excel(path /'TEST'       / 'TEST_QUANTITATIVE_METADATA.xlsx')    
    tsf=pd.read_csv(path   /'TEST'       / 'TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')    
    sub=pd.read_excel(path / 'SAMPLE_SUBMISSION.xlsx')
    dic=pd.read_excel(path /'Data Dictionary.xlsx')
    return trc, trq, trf, trs, tsc, tsq, tsf, sub, dic

trc, trq, trf, trs, tsc, tsq, tsf, sub, dic = read_data(base_path=path)

# Data Merging 
cq = pd.merge(trc, trq, on='participant_id', how='left')
feat = pd.merge(cq, trf, on='participant_id', how='left')  
qc = pd.merge(tsc, tsq, on='participant_id', how='left')
train = pd.merge(feat, trs, on='participant_id', how='left') 
test = pd.merge(qc, tsf, on='participant_id', how='left')

# Adding Features before Preprocessing

In [ ]:
# Your coefficients (estimated from your description)
female_coefs = {
    'ColorVision_CV_Score': 2.900,
    'SDQ_SDQ_Emotional_Problems': 2.814,
    'SDQ_SDQ_Prosocial': 2.661,
    'SDQ_SDQ_Internalizing': 1.382,
    'SDQ_SDQ_Conduct_Problems': 1.329,
    'SDQ_SDQ_Generating_Impact': 1.109,
    'APQ_P_APQ_P_PM': 0.642
}

male_coefs = {
    'SDQ_SDQ_Hyperactivity': -3.088,
    'APQ_P_APQ_P_PP': -1.903,
    'SDQ_SDQ_Externalizing': -1.447,
    'APQ_P_APQ_P_CP': -1.380,
    'APQ_P_APQ_P_INV': -0.437,
    'APQ_P_APQ_P_OPD': -0.676,
    'SDQ_SDQ_Peer_Problems': -0.596
}

# Compute weighted sums
train['female_symptom_score'] = sum(train[feat] * coef for feat, coef in female_coefs.items())
train['male_symptom_score'] = sum(train[feat] * coef for feat, coef in male_coefs.items())

test['female_symptom_score'] = sum(test[feat] * coef for feat, coef in female_coefs.items())
test['male_symptom_score'] = sum(test[feat] * coef for feat, coef in male_coefs.items())

# Preprocessing

### Drop Irrelevant Columns

In [ ]:
cols_to_drop = [
    'MRI_Track_Age_at_Scan', 'Basic_Demos_Enroll_Year', 'Basic_Demos_Study_Site',
    'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race',
    'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu',
    'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu',
    'Barratt_Barratt_P2_Occ', 'EHQ_EHQ_Total'
]

train_ids = train['participant_id']
test_ids = test['participant_id'] # I will store them for later usage in grouping in validation why?  I don't want the same user to appear in both train and test. 


for df in (train, test):
    df.set_index('participant_id', inplace=True)
    df.drop(columns=cols_to_drop, errors='ignore', inplace=True)

# for df in (train,test):
#     # df.drop(columns=['participant_id'], inplace=True) 
#     df.set_index('participant_id', inplace=True)
#     df.drop(columns=['EHQ_EHQ_Total'], inplace=True) 
#     df.drop(columns=['MRI_Track_Age_at_Scan'], inplace=True) 

### Setup

In [ ]:
# Highlighting important variables. 
# note that I did't deal with the quantative data trq as categorical
# I will use the OneHotEncoder for the categorical data as we have some data trap that I don't think we can use the label encoder for. 

# for feature in trc.columns:
#     train[feature] = train[feature].astype(object)
num_feats = [feature for feature in train.columns if train[feature].dtype == 'float64']
cat_feats = [feature for feature in train.columns if train[feature].dtype == 'object'] # seperate categorical and numerical features help me reteriving them later easily for preprocessing.
target_cols = ['ADHD_Outcome', 'Sex_F']

### Log Transformation

In [ ]:
# Features with positive skewness and non-negative values: apply log transformation
# num_feats[:18] Don't transform fMRI data because it has negative values
log_features = [f for f in num_feats[:18] if (train[f] >= 0).all() and scipy.stats.skew(train[f]) > 0.5]  # Only apply to features with significant positive skew

# Apply log transformation for the selected features
for feature in log_features:
    train[feature] = np.log1p(train[feature])  # Apply log(x+1) to handle skewed features
    test[feature] = np.log1p(test[feature])    # Apply log(x+1) to test data as well


### Missing Values

In [ ]:
# Find columns with missing values only
train_missing_features_to_impute = train.columns[train.isnull().any()].tolist() # List of features with missing values in train, only 25 and no missing data in fMRI data
test_missing_features_to_impute = test.columns[test.isnull().any()].tolist() # List of features with missing values in test, only 23 and no missing data in fMRI data

# Initialize the imputer
imputer = IterativeImputer(estimator=LassoCV(random_state=42), max_iter=5, random_state=42)

# Impute in-place
train[train_missing_features_to_impute] = imputer.fit_transform(train[train_missing_features_to_impute])
test[test_missing_features_to_impute] = imputer.fit_transform(test[test_missing_features_to_impute])

### Encode Categorical Features 

In [ ]:
"""all categorical features have been dropped so no need for this."""

# # Convert all categorical features to strings (to avoid mixed types)
# for feature in cat_feats:
#     train[feature] = train[feature].astype(str)
#     test[feature] = test[feature].astype(str)

# # One-Hot Encoding for categorical features
# encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# for feature in cat_feats:
#     if feature == 'participant_id':  # Skip participant_id since it's not a feature
#         continue

#     # Apply OneHotEncoder
#     train_encoded = encoder.fit_transform(train[[feature]])
#     test_encoded = encoder.transform(test[[feature]])

#     # Convert encoded features to DataFrame and append them to the original data
#     train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out([feature]))
#     test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out([feature]))

#     # Drop the original feature and concatenate the new encoded columns
#     train = pd.concat([train.drop(columns=[feature]), train_encoded_df], axis=1)
#     test = pd.concat([test.drop(columns=[feature]), test_encoded_df], axis=1)

'all categorical features have been dropped so no need for this.'

### Scale Numerical Features

In [ ]:
scaler = StandardScaler()

# Only apply scaling to numerical columns that are not part of the target or categorical features
numerical_features = [col for col in train.columns if col not in target_cols and col not in cat_feats]

# Fit scaler on the numerical features of the train set and transform train and test sets
train[numerical_features] = scaler.fit_transform(train[numerical_features])  # Fit and transform for train set
test[numerical_features] = scaler.transform(test[numerical_features])        # Only transform for test set (avoid data leakage)

In [ ]:
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

print(non_connectome_features)
print(len(non_connectome_features))

['ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'female_symptom_score', 'male_symptom_score']
18


# Training and Validation

### All data (fMRI and symtpoms) used

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# Constants
SEED = 7
N_SPLITS = 5
ADHD_WEIGHT = 2  # 2x weight for female ADHD cases

non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models with competition-optimized parameters
model_sex = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Configure cross-validation
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Store competition metrics
adhd_f1_scores = []
sex_f1_scores = []

# Define important features for interaction (update with your actual features)
interaction_features = [    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
                        "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score']  # Use actual column names

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['ADHD_Outcome']), 1):
    print(f"\n=== Fold {fold} ===")
    
    # Split data
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    
    # Extract targets
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Calculate competition weights
    train_weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) * (ADHD_WEIGHT - 1) + 1
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) * (ADHD_WEIGHT - 1) + 1

    # --- Sex Prediction ---
    # Train sex model
    model_sex.fit(X_train.drop(columns=['Sex_F', 'ADHD_Outcome']), y_train_sex, sample_weight=train_weights)
    
    # Get sex probabilities
    sex_train_proba = model_sex.predict_proba(X_train.drop(columns=['Sex_F', 'ADHD_Outcome']))[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val.drop(columns=['Sex_F', 'ADHD_Outcome']))[:, 1]

    # --- ADHD Prediction with Interaction Features ---
    # Create enhanced features
    X_train_adhd = X_train.drop(columns=['Sex_F', 'ADHD_Outcome']).copy()
    X_val_adhd = X_val.drop(columns=['Sex_F', 'ADHD_Outcome']).copy()
    
    # Add sex probability feature
    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba
    
    # Add interaction terms
    for col in interaction_features:
        X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
        X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    # Train ADHD model
    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)
    
    # --- Competition Validation ---
    sex_pred = model_sex.predict(X_val.drop(columns=['Sex_F', 'ADHD_Outcome']))
    adhd_pred = model_adhd.predict(X_val_adhd)
    
    # Calculate weighted F1 scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# --- Final Competition Scoring ---
print("\n=== Competition Results ===")
print(f"Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# Calculate final leaderboard score (average of both F1 scores)
final_score = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
print(f"\nCompetition Score (Normal Mean): {final_score:.4f}")

print(f"Mean Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Mean ADHD_Outcome F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# Calculate final leaderboard score (harmonic mean of average F1s)
final_score = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
print(f"\nCompetition Score (Harmonic Mean): {final_score:.4f}")


=== Fold 1 ===
Sex_F F1: 0.7391
ADHD_Outcome F1: 0.4228

=== Fold 2 ===
Sex_F F1: 0.7345
ADHD_Outcome F1: 0.4407

=== Fold 3 ===
Sex_F F1: 0.6424
ADHD_Outcome F1: 0.4536

=== Fold 4 ===
Sex_F F1: 0.6984
ADHD_Outcome F1: 0.4429

=== Fold 5 ===
Sex_F F1: 0.7077
ADHD_Outcome F1: 0.4737

=== Competition Results ===
Sex_F F1: 0.7044 ± 0.0346
ADHD F1: 0.4467 ± 0.0167

Competition Score (Normal Mean): 0.5756
Mean Sex_F F1: 0.7044 ± 0.0346
Mean ADHD_Outcome F1: 0.4467 ± 0.0167

Competition Score (Harmonic Mean): 0.5467


### All data (fMRI and symtpoms) used, added more female symptoms features to interaction_features

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# Constants
SEED = 7
N_SPLITS = 5

non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models with competition-optimized parameters
model_sex = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Configure cross-validation
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Store competition metrics
adhd_f1_scores = []
sex_f1_scores = []

# Define important features for interaction (update with your actual features)
interaction_features = [ "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
                        "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score', 
                        'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems', 
                        'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'] 

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['ADHD_Outcome']), 1):
    print(f"\n=== Fold {fold} ===")
    
    # Split data
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    
    # Extract targets
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Calculate competition weights
    train_weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) + 1
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

    # --- Sex Prediction ---
    # Train sex model
    model_sex.fit(X_train.drop(columns=['Sex_F', 'ADHD_Outcome']), y_train_sex, sample_weight=train_weights)
    
    # Get sex probabilities
    sex_train_proba = model_sex.predict_proba(X_train.drop(columns=['Sex_F', 'ADHD_Outcome']))[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val.drop(columns=['Sex_F', 'ADHD_Outcome']))[:, 1]

    # --- ADHD Prediction with Interaction Features ---
    # Create enhanced features
    X_train_adhd = X_train.drop(columns=['Sex_F', 'ADHD_Outcome']).copy()
    X_val_adhd = X_val.drop(columns=['Sex_F', 'ADHD_Outcome']).copy()
    
    # Add sex probability feature
    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba
    
    # Add interaction terms
    for col in interaction_features:
        X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
        X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    # Train ADHD model
    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)
    
    # --- Competition Validation ---
    sex_pred = model_sex.predict(X_val.drop(columns=['Sex_F', 'ADHD_Outcome']))
    adhd_pred = model_adhd.predict(X_val_adhd)
    
    # Calculate weighted F1 scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# --- Final Competition Scoring ---
print("\n=== Competition Results ===")
print(f"Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# Calculate final leaderboard score (average of both F1 scores)
final_score = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
print(f"\nCompetition Score (Normal Mean): {final_score:.4f}")

print(f"Mean Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Mean ADHD_Outcome F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# Calculate final leaderboard score (harmonic mean of average F1s)
final_score = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
print(f"\nCompetition Score (Harmonic Mean): {final_score:.4f}")


=== Fold 1 ===
Sex_F F1: 0.7391
ADHD_Outcome F1: 0.4228

=== Fold 2 ===
Sex_F F1: 0.7345
ADHD_Outcome F1: 0.4512

=== Fold 3 ===
Sex_F F1: 0.6424
ADHD_Outcome F1: 0.4642

=== Fold 4 ===


KeyboardInterrupt: 

### Symptoms only used (no fMRI), added more female symptoms features to interaction_features same as above

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# Constants
SEED = 7
N_SPLITS = 5

# Restrict to non-connectome features
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models
model_sex = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Cross-validation setup
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Score containers
adhd_f1_scores = []
sex_f1_scores = []

# Features used in interaction
interaction_features = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
    "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score', 
    'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems', 
    'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'
] 

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['ADHD_Outcome']), 1):
    print(f"\n=== Fold {fold} ===")

    X_train_full, X_val_full = train.iloc[train_idx], train.iloc[val_idx]
    
    # Targets
    y_train_sex = X_train_full['Sex_F']
    y_val_sex = X_val_full['Sex_F']
    y_train_adhd = X_train_full['ADHD_Outcome']
    y_val_adhd = X_val_full['ADHD_Outcome']

    # Sample weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1

    # Restrict features to non-connectome
    X_train = X_train_full[non_connectome_features].copy()
    X_val = X_val_full[non_connectome_features].copy()

    # --- Sex Model ---
    model_sex.fit(X_train, y_train_sex, sample_weight=train_weights)
    sex_train_proba = model_sex.predict_proba(X_train)[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val)[:, 1]

    # --- ADHD Model ---
    X_train_adhd = X_train.copy()
    X_val_adhd = X_val.copy()

    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba

    # Add interaction features
    for col in interaction_features:
        if col in X_train_adhd.columns:
            X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
            X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)

    # Predict
    sex_pred = model_sex.predict(X_val)
    adhd_pred = model_adhd.predict(X_val_adhd)

    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Final results
print("\n=== Competition Results ===")
print(f"Average Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Average ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# NEW: Calculate harmonic mean per fold first
per_fold_hmeans = [hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)]
final_score_hmean_foldwise = np.mean(per_fold_hmeans)

# Original approach for comparison
final_score_mean = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
final_score_hmean = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])

print(f"\nCompetition Score (Normal Mean): {final_score_mean:.4f}")
print(f"Competition Score (Harmonic Mean of Averages): {final_score_hmean:.4f}")
print(f"Competition Score (Average of Harmonic Means): {final_score_hmean_foldwise:.4f}")  # This is likely what leaderboard uses



=== Fold 1 ===
Sex_F F1: 0.4407
ADHD_Outcome F1: 0.8894

=== Fold 2 ===
Sex_F F1: 0.4978
ADHD_Outcome F1: 0.8856

=== Fold 3 ===
Sex_F F1: 0.5415
ADHD_Outcome F1: 0.8486

=== Fold 4 ===
Sex_F F1: 0.5250
ADHD_Outcome F1: 0.8700

=== Fold 5 ===
Sex_F F1: 0.5804
ADHD_Outcome F1: 0.8874

=== Competition Results ===
Average Sex_F F1: 0.5171 ± 0.0466
Average ADHD F1: 0.8762 ± 0.0154

Competition Score (Normal Mean): 0.6966
Competition Score (Harmonic Mean of Averages): 0.6503
Competition Score (Average of Harmonic Means): 0.6489


### Stratify based on Sex_F

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# Constants
SEED = 7
N_SPLITS = 5

# Restrict to non-connectome features
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models
model_sex = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Cross-validation setup
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Score containers
adhd_f1_scores = []
sex_f1_scores = []

# Features used in interaction
interaction_features = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
    "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score', 
    'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems', 
    'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'
] 

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['Sex_F']), 1):
    print(f"\n=== Fold {fold} ===")

    X_train_full, X_val_full = train.iloc[train_idx], train.iloc[val_idx]
    
    # Targets
    y_train_sex = X_train_full['Sex_F']
    y_val_sex = X_val_full['Sex_F']
    y_train_adhd = X_train_full['ADHD_Outcome']
    y_val_adhd = X_val_full['ADHD_Outcome']

    # Sample weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1

    # Restrict features to non-connectome
    X_train = X_train_full[non_connectome_features].copy()
    X_val = X_val_full[non_connectome_features].copy()

    # --- Sex Model ---
    model_sex.fit(X_train, y_train_sex, sample_weight=train_weights)
    sex_train_proba = model_sex.predict_proba(X_train)[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val)[:, 1]

    # --- ADHD Model ---
    X_train_adhd = X_train.copy()
    X_val_adhd = X_val.copy()

    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba

    # Add interaction features
    for col in interaction_features:
        if col in X_train_adhd.columns:
            X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
            X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)

    # Predict
    sex_pred = model_sex.predict(X_val)
    adhd_pred = model_adhd.predict(X_val_adhd)

    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Final results
print("\n=== Competition Results ===")
print(f"Average Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Average ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# NEW: Calculate harmonic mean per fold first
per_fold_hmeans = [hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)]
final_score_hmean_foldwise = np.mean(per_fold_hmeans)

# Original approach for comparison
final_score_mean = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
final_score_hmean = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])

print(f"\nCompetition Score (Normal Mean): {final_score_mean:.4f}")
print(f"Competition Score (Harmonic Mean of Averages): {final_score_hmean:.4f}")
print(f"Competition Score (Average of Harmonic Means): {final_score_hmean_foldwise:.4f}")  # This is likely what leaderboard uses



=== Fold 1 ===
Sex_F F1: 0.5344
ADHD_Outcome F1: 0.8856

=== Fold 2 ===
Sex_F F1: 0.5225
ADHD_Outcome F1: 0.8756

=== Fold 3 ===
Sex_F F1: 0.5316
ADHD_Outcome F1: 0.8700

=== Fold 4 ===
Sex_F F1: 0.5103
ADHD_Outcome F1: 0.8973

=== Fold 5 ===
Sex_F F1: 0.5299
ADHD_Outcome F1: 0.8889

=== Competition Results ===
Average Sex_F F1: 0.5258 ± 0.0087
Average ADHD F1: 0.8835 ± 0.0097

Competition Score (Normal Mean): 0.7046
Competition Score (Harmonic Mean of Averages): 0.6592
Competition Score (Average of Harmonic Means): 0.6591


### C=10 for Sex, Stratify on Sex, No fMRI

In [ ]:
# Constants
SEED = 7
N_SPLITS = 5
target_cols = ['ADHD_Outcome', 'Sex_F']  # Define target columns

# Define non-connectome features
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models
model_sex = LogisticRegression(
    penalty='l2',
    C=10,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Cross-validation setup
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Score containers
F1s = []
stds = []

for fold_number, (train_index, test_index) in enumerate(cv.split(train[non_connectome_features], train['Sex_F']), 1):
    print(f"\n==== Fold {fold_number} ===")
    
    train_v = train.iloc[train_index]
    test_v = train.iloc[test_index]
    
    # Sample weights
    weights = ((train_v['Sex_F'] == 1) & (train_v['ADHD_Outcome'] == 1)).astype(int) + 1

    # --- Sex Model ---
    model_sex.fit(
        train_v[non_connectome_features], 
        train_v['Sex_F'],
        sample_weight=weights
    )
    sex_pred = model_sex.predict(test_v[non_connectome_features])

    # --- ADHD Model ---
    model_adhd.fit(
        train_v[non_connectome_features],
        train_v['ADHD_Outcome'],
        sample_weight=weights
    )
    adhd_pred = model_adhd.predict(test_v[non_connectome_features])

    # Validation
    valid_idx = test_v[target_cols].notna().all(axis=1)
    valid_testset = test_v.loc[valid_idx, target_cols]

    sex_f1 = f1_score(valid_testset['Sex_F'], sex_pred[valid_idx])
    adhd_f1 = f1_score(valid_testset['ADHD_Outcome'], adhd_pred[valid_idx])

    print(f"Sex_F -> F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome -> F1: {adhd_f1:.4f}")

    F1s.append([adhd_f1, sex_f1])
    stds.append(test_v[target_cols].std())

# Final scoring
F1s = np.array(F1s)
mean_f1_scores = F1s.mean(axis=0)
print("\n==== Overall Results ===")
print("Mean F1-scores:")
for i, target in enumerate(target_cols):
    print(f"    {target} -> F1: {mean_f1_scores[i]:.4f}")

f1_stds = F1s.std(axis=0)
print("\nF1-score stds:")
for i, target in enumerate(target_cols):
    print(f"    {target} -> Std: {f1_stds[i]:.4f}")

print("\nStandard deviations of test sets:")
for std in stds:
    print(f"    {std.to_dict()}")

# Final harmonic mean score
print("\nscore mean (ADHD Sex): ", np.mean(F1s, axis=0))
score = hmean(F1s, axis=0)  # Harmonic mean per fold
score = hmean(score)        # Then overall harmonic mean
print("\nFinal Overall Harmonic Mean (Mimic Leaderboard): ", score)



==== Fold 1 ===
Sex_F -> F1: 0.4910
ADHD_Outcome -> F1: 0.8681

==== Fold 2 ===
Sex_F -> F1: 0.4819
ADHD_Outcome -> F1: 0.8475

==== Fold 3 ===
Sex_F -> F1: 0.4734
ADHD_Outcome -> F1: 0.8523

==== Fold 4 ===
Sex_F -> F1: 0.4767
ADHD_Outcome -> F1: 0.8753

==== Fold 5 ===
Sex_F -> F1: 0.4790
ADHD_Outcome -> F1: 0.8644

==== Overall Results ===
Mean F1-scores:
    ADHD_Outcome -> F1: 0.8615
    Sex_F -> F1: 0.4804

F1-score stds:
    ADHD_Outcome -> Std: 0.0103
    Sex_F -> Std: 0.0060

Standard deviations of test sets:
    {'ADHD_Outcome': 0.46115656157267876, 'Sex_F': 0.4752128960101643}
    {'ADHD_Outcome': 0.47788916775847196, 'Sex_F': 0.4752128960101643}
    {'ADHD_Outcome': 0.4693800748445646, 'Sex_F': 0.47657075200345245}
    {'ADHD_Outcome': 0.44628480198336107, 'Sex_F': 0.475687132835641}
    {'ADHD_Outcome': 0.46833112435802715, 'Sex_F': 0.475687132835641}

score mean (ADHD Sex):  [0.86152011 0.48042091]

Final Overall Harmonic Mean (Mimic Leaderboard):  0.6167636839329982


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# Constants
SEED = 7
N_SPLITS = 5

# Restrict to non-connectome features
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]

# Initialize models
model_sex = LogisticRegression(
    penalty='l2',
    C=10,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Cross-validation setup
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Score containers
adhd_f1_scores = []
sex_f1_scores = []

# Features used in interaction
interaction_features = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
    "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score', 
    'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems', 
    'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'
] 

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['Sex_F']), 1):
    print(f"\n=== Fold {fold} ===")

    X_train_full, X_val_full = train.iloc[train_idx], train.iloc[val_idx]
    
    # Targets
    y_train_sex = X_train_full['Sex_F']
    y_val_sex = X_val_full['Sex_F']
    y_train_adhd = X_train_full['ADHD_Outcome']
    y_val_adhd = X_val_full['ADHD_Outcome']

    # Sample weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1

    # Restrict features to non-connectome
    X_train = X_train_full[non_connectome_features].copy()
    X_val = X_val_full[non_connectome_features].copy()

    # --- Sex Model ---
    model_sex.fit(X_train, y_train_sex, sample_weight=train_weights)
    sex_train_proba = model_sex.predict_proba(X_train)[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val)[:, 1]

    # --- ADHD Model ---
    X_train_adhd = X_train.copy()
    X_val_adhd = X_val.copy()

    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba

    # Add interaction features
    for col in interaction_features:
        if col in X_train_adhd.columns:
            X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
            X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)

    # Predict
    sex_pred = model_sex.predict(X_val)
    adhd_pred = model_adhd.predict(X_val_adhd)

    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Final results
print("\n=== Competition Results ===")
print(f"Average Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Average ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# NEW: Calculate harmonic mean per fold first
per_fold_hmeans = [hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)]
final_score_hmean_foldwise = np.mean(per_fold_hmeans)

# Original approach for comparison
final_score_mean = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
final_score_hmean = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])

print(f"\nCompetition Score (Normal Mean): {final_score_mean:.4f}")
print(f"Competition Score (Harmonic Mean of Averages): {final_score_hmean:.4f}")
print(f"Competition Score (Average of Harmonic Means): {final_score_hmean_foldwise:.4f}")  # This is likely what leaderboard uses



=== Fold 1 ===
Sex_F F1: 0.5418
ADHD_Outcome F1: 0.8856

=== Fold 2 ===
Sex_F F1: 0.5286
ADHD_Outcome F1: 0.8756

=== Fold 3 ===
Sex_F F1: 0.5246
ADHD_Outcome F1: 0.8700

=== Fold 4 ===
Sex_F F1: 0.5301
ADHD_Outcome F1: 0.8973

=== Fold 5 ===
Sex_F F1: 0.5272
ADHD_Outcome F1: 0.8889

=== Competition Results ===
Average Sex_F F1: 0.5305 ± 0.0060
Average ADHD F1: 0.8835 ± 0.0097

Competition Score (Normal Mean): 0.7070
Competition Score (Harmonic Mean of Averages): 0.6629
Competition Score (Average of Harmonic Means): 0.6629


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# ========================
# YOUR APPROACH (ORIGINAL)
# ========================
print("\n" + "="*40)
print("RUNNING YOUR ORIGINAL APPROACH")
print("="*40 + "\n")

# [Rest of your original code up to the final scoring]
# ... [All your original code remains unchanged until the final scoring]

# Final results - Your approach
print("\n=== YOUR FINAL RESULTS ===")
print(f"Average Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Average ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# Your scoring methods
per_fold_hmeans = [hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)]
final_score_hmean_foldwise = np.mean(per_fold_hmeans)
final_score_mean = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
final_score_hmean = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])

print(f"\nYOUR Competition Score (Normal Mean): {final_score_mean:.4f}")
print(f"YOUR Competition Score (Harmonic Mean of Averages): {final_score_hmean:.4f}")
print(f"YOUR Competition Score (Average of Harmonic Means): {final_score_hmean_foldwise:.4f}")

# ========================
# AHMED'S APPROACH 
# ========================
print("\n" + "="*40)
print("RUNNING AHMED'S APPROACH")
print("="*40 + "\n")

# Re-initialize models for Ahmed's approach
model_sex_ahmed = LogisticRegression(
    penalty='l2',
    C=0.02,  # Using Ahmed's C value
    random_state=SEED,
    solver='saga',
    max_iter=1000,
    class_weight='balanced'  # Ahmed's addition
)

model_adhd_ahmed = LogisticRegression(
    penalty='l2',
    C=0.02,  # Using Ahmed's C value
    random_state=SEED,
    solver='saga',
    max_iter=1000,
    class_weight='balanced'  # Ahmed's addition
)

# Ahmed's score containers
F1s = []
stds = []

for fold_number, (train_index, test_index) in enumerate(cv.split(train[non_connectome_features], train['ADHD_Outcome']), 1):
    print(f"\n==== Fold {fold_number} === (Ahmed's Approach)")
    
    train_v = train.iloc[train_index]
    test_v = train.iloc[test_index]
    
    # Ahmed's weight calculation (same as yours)
    weights = ((train_v['Sex_F'] == 1) & (train_v['ADHD_Outcome'] == 1)).astype(int) + 1

    # --- Sex Model ---
    model_sex_ahmed.fit(
        train_v[non_connectome_features], 
        train_v['Sex_F'],
        sample_weight=weights
    )
    sex_pred = model_sex_ahmed.predict(test_v[non_connectome_features])

    # --- ADHD Model ---
    model_adhd_ahmed.fit(
        train_v[non_connectome_features],
        train_v['ADHD_Outcome'],
        sample_weight=weights
    )
    adhd_pred = model_adhd_ahmed.predict(test_v[non_connectome_features])

    # Ahmed's validation with null check
    valid_idx = test_v[target_cols].notna().all(axis=1)
    valid_testset = test_v.loc[valid_idx, target_cols]

    sex_f1 = f1_score(valid_testset['Sex_F'], sex_pred[valid_idx])
    adhd_f1 = f1_score(valid_testset['ADHD_Outcome'], adhd_pred[valid_idx])

    print(f"Sex_F -> F1: {sex_f1:.4f} (Ahmed)")
    print(f"ADHD_Outcome -> F1: {adhd_f1:.4f} (Ahmed)")

    F1s.append([adhd_f1, sex_f1])
    stds.append(test_v[target_cols].std())

# Ahmed's final scoring
F1s = np.array(F1s)
mean_f1_scores = F1s.mean(axis=0)
print("\n==== AHMED'S OVERALL RESULTS ====")
print("Mean F1-scores:")
for i, target in enumerate(target_cols):
    print(f"    {target} -> F1: {mean_f1_scores[i]:.4f}")

# ========================
# YOUR APPROACH 
# ========================
print("\n=== RUNNING YOUR APPROACH ===\n")

# [Your original code execution...]

your_results = {
    "Sex_F F1 (mean)": np.mean(sex_f1_scores),
    "ADHD F1 (mean)": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Harmonic Mean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Average of Harmonic Means": np.mean([hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)])
}

# ========================
# AHMED'S APPROACH 
# ========================
print("\n=== RUNNING AHMED'S APPROACH ===\n")

# [Ahmed's code execution...]

ahmed_results = {
    "Sex_F F1 (mean)": F1s[:,1].mean(),
    "ADHD F1 (mean)": F1s[:,0].mean(),
    "Normal Mean": np.mean(F1s.mean(axis=0)),
    "Harmonic Mean of Averages": hmean(F1s.mean(axis=0)),
    "Average of Harmonic Means": np.mean([hmean(fold) for fold in F1s]),
    "Double Harmonic Mean": hmean(hmean(F1s, axis=0))
}
# ========================
# SIDE-BY-SIDE COMPARISON
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<30} | {:>15} | {:>15}".format("Metric", "Your Approach", "Ahmed's Approach"))
print("-" * 70)
for key in your_results:
    if key in ahmed_results:
        print("{:<30} | {:>15.4f} | {:>15.4f}".format(key, your_results[key], ahmed_results[key]))
# Show Ahmed's special score separately



RUNNING YOUR ORIGINAL APPROACH


=== YOUR FINAL RESULTS ===
Average Sex_F F1: 0.7010 ± 0.0189
Average ADHD F1: 0.4598 ± 0.0263

YOUR Competition Score (Normal Mean): 0.5804
YOUR Competition Score (Harmonic Mean of Averages): 0.5554
YOUR Competition Score (Average of Harmonic Means): 0.5545

RUNNING AHMED'S APPROACH


==== Fold 1 === (Ahmed's Approach)
Sex_F -> F1: 0.5467 (Ahmed)
ADHD_Outcome -> F1: 0.8250 (Ahmed)

==== Fold 2 === (Ahmed's Approach)
Sex_F -> F1: 0.5782 (Ahmed)
ADHD_Outcome -> F1: 0.8418 (Ahmed)

==== Fold 3 === (Ahmed's Approach)
Sex_F -> F1: 0.4552 (Ahmed)
ADHD_Outcome -> F1: 0.7925 (Ahmed)

==== Fold 4 === (Ahmed's Approach)
Sex_F -> F1: 0.5072 (Ahmed)
ADHD_Outcome -> F1: 0.8483 (Ahmed)

==== Fold 5 === (Ahmed's Approach)
Sex_F -> F1: 0.5017 (Ahmed)
ADHD_Outcome -> F1: 0.8344 (Ahmed)

==== AHMED'S OVERALL RESULTS ====
Mean F1-scores:
    ADHD_Outcome -> F1: 0.8284
    Sex_F -> F1: 0.5178

=== RUNNING YOUR APPROACH ===


=== RUNNING AHMED'S APPROACH ===


=== FINAL CO

In [ ]:

# ========================
# YOUR APPROACH 
# ========================
print("\n=== RUNNING YOUR APPROACH ===\n")

# [Your original code execution...]

your_results = {
    "Sex_F F1 (mean)": np.mean(sex_f1_scores),
    "ADHD F1 (mean)": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Harmonic Mean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Average of Harmonic Means": np.mean([hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)])
}

# ========================
# AHMED'S APPROACH 
# ========================
print("\n=== RUNNING AHMED'S APPROACH ===\n")

# [Ahmed's code execution...]

ahmed_results = {
    "Sex_F F1 (mean)": F1s[:,1].mean(),
    "ADHD F1 (mean)": F1s[:,0].mean(),
    "Normal Mean": np.mean(F1s.mean(axis=0)),
    "Harmonic Mean of Averages": hmean(F1s.mean(axis=0)),
    "Average of Harmonic Means": np.mean([hmean(fold) for fold in F1s]),
    "Double Harmonic Mean": hmean(hmean(F1s, axis=0))
}
# ========================
# SIDE-BY-SIDE COMPARISON
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<30} | {:>15} | {:>15}".format("Metric", "Your Approach", "Ahmed's Approach"))
print("-" * 70)
for key in your_results:
    if key in ahmed_results:
        print("{:<30} | {:>15.4f} | {:>15.4f}".format(key, your_results[key], ahmed_results[key]))
# Show Ahmed's special score separately



=== RUNNING YOUR APPROACH ===


=== RUNNING AHMED'S APPROACH ===


=== FINAL COMPARISON ===
Metric                         |   Your Approach | Ahmed's Approach
----------------------------------------------------------------------
Sex_F F1 (mean)                |          0.7010 |          0.5178
ADHD F1 (mean)                 |          0.4598 |          0.8284
Normal Mean                    |          0.5804 |          0.6731
Harmonic Mean of Averages      |          0.5554 |          0.6373
Average of Harmonic Means      |          0.5545 |          0.6366


### train ADHD model on symptoms only, sex model on all data or fMRI only

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# Constants
SEED = 7
N_SPLITS = 5
target_cols = ['ADHD_Outcome', 'Sex_F']

# Feature selection
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]
# ========================
# YOUR APPROACH (ADJUSTED)
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

# [Previous initialization code remains the same...]

# Key fix: Use the same features for stratification as for training
full_features = [col for col in train.columns if col not in target_cols]

# Cross-validation - stratify on the full features
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores = []
sex_f1_scores = []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    print(f"\n=== Fold {fold} (Adjusted) ===")
    
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1
    
    # --- Sex model: use ALL features ---
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=train_weights)
    sex_pred = model_sex.predict(X_val[full_features])
    
    # --- ADHD model: use only non-connectome features ---
    model_adhd.fit(X_train[non_connectome_features], y_train_adhd, sample_weight=train_weights)
    adhd_pred = model_adhd.predict(X_val[non_connectome_features])
    
    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Your adjusted results
your_adjusted_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "HMean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Avg of HMeans": np.mean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)])
}

# [Rest of the code remains the same...]




# ========================
# AHMED'S APPROACH (ADJUSTED)
# ========================
print("\n=== AHMED'S APPROACH (ADJUSTED) ===\n")

# Initialize Ahmed's models with your requested changes
model_sex_ahmed = LogisticRegression(
    penalty='l2', 
    C=10,  # Changed from 0.02 to 10 for sex model
    random_state=SEED,
    solver='saga', 
    max_iter=1000,
    # Removed class_weight='balanced'
)

model_adhd_ahmed = LogisticRegression(
    penalty='l2', 
    C=0.02,  # Kept original C value for ADHD
    random_state=SEED,
    solver='saga', 
    max_iter=1000,
    # Removed class_weight='balanced'
)

# Cross-validation - now stratifying by Sex_F
F1s = []
for fold, (train_idx, val_idx) in enumerate(cv.split(train[non_connectome_features], train['Sex_F']), 1):
    print(f"\n=== Fold {fold} (Ahmed Adjusted) ===")
    
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) + 1
    
    # --- Sex model: use full feature set ---
    full_features = [col for col in train.columns if col not in target_cols]
    model_sex_ahmed.fit(X_train[full_features], X_train['Sex_F'], sample_weight=weights)
    sex_pred = model_sex_ahmed.predict(X_val[full_features])
    
    # --- ADHD model: use non-connectome features ---
    model_adhd_ahmed.fit(X_train[non_connectome_features], X_train['ADHD_Outcome'], sample_weight=weights)
    adhd_pred = model_adhd_ahmed.predict(X_val[non_connectome_features])
    
    # Validation
    valid_idx = X_val[target_cols].notna().all(axis=1)
    sex_f1 = f1_score(X_val.loc[valid_idx, 'Sex_F'], sex_pred[valid_idx])
    adhd_f1 = f1_score(X_val.loc[valid_idx, 'ADHD_Outcome'], adhd_pred[valid_idx])
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    F1s.append([adhd_f1, sex_f1])

F1s = np.array(F1s)

# Adjusted results without Double Harmonic Mean
ahmed_adjusted_results = {
    "Sex_F F1": F1s[:,1].mean(),
    "ADHD F1": F1s[:,0].mean(),
    "Normal Mean": np.mean(F1s.mean(axis=0)),
    "HMean of Averages": hmean(F1s.mean(axis=0)),
    "Avg of HMeans": np.mean([hmean(fold) for fold in F1s])
    # Removed "Double HMean"
}

# ========================
# UPDATED COMPARISON
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<30} | {:>15} | {:>15}".format("Metric", "Your Approach", "Ahmed's Adjusted"))
print("-" * 70)
for key in your_results:
    if key in ahmed_adjusted_results:
        print("{:<30} | {:>15.4f} | {:>15.4f}".format(key, your_results[key], ahmed_adjusted_results[key]))


=== YOUR APPROACH (ADJUSTED) ===


=== Fold 1 (Adjusted) ===
Sex_F F1: 0.7221
ADHD_Outcome F1: 0.8856

=== Fold 2 (Adjusted) ===
Sex_F F1: 0.6894
ADHD_Outcome F1: 0.8736

=== Fold 3 (Adjusted) ===
Sex_F F1: 0.7251
ADHD_Outcome F1: 0.8680

=== Fold 4 (Adjusted) ===
Sex_F F1: 0.6783
ADHD_Outcome F1: 0.8977

=== Fold 5 (Adjusted) ===
Sex_F F1: 0.6901
ADHD_Outcome F1: 0.8889

=== AHMED'S APPROACH (ADJUSTED) ===


=== Fold 1 (Ahmed Adjusted) ===
Sex_F F1: 0.6167
ADHD_Outcome F1: 0.8681

=== Fold 2 (Ahmed Adjusted) ===
Sex_F F1: 0.6066
ADHD_Outcome F1: 0.8475

=== Fold 3 (Ahmed Adjusted) ===
Sex_F F1: 0.6441
ADHD_Outcome F1: 0.8523

=== Fold 4 (Ahmed Adjusted) ===
Sex_F F1: 0.5749
ADHD_Outcome F1: 0.8753

=== Fold 5 (Ahmed Adjusted) ===
Sex_F F1: 0.5887
ADHD_Outcome F1: 0.8644

=== FINAL COMPARISON ===
Metric                         |   Your Approach | Ahmed's Adjusted
----------------------------------------------------------------------
Sex_F F1                       |          0.5305 |  

In [ ]:
# Adding double hmean
# ========================
# YOUR APPROACH (ADJUSTED)
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

# [Previous code remains identical until the results section...]

# Your adjusted results - now with teammate's scoring method
your_adjusted_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "HMean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Avg of HMeans": np.mean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)]),
    # Teammate's scoring method added:
    "Double HMean": hmean(hmean(np.column_stack([sex_f1_scores, adhd_f1_scores]), axis=0))
}

# ========================
# AHMED'S APPROACH (ADJUSTED)
# ========================
print("\n=== AHMED'S APPROACH (ADJUSTED) ===\n")

# [Previous code remains identical until the results section...]

# Ahmed's adjusted results - now with teammate's scoring method
ahmed_adjusted_results = {
    "Sex_F F1": F1s[:,1].mean(),
    "ADHD F1": F1s[:,0].mean(),
    "Normal Mean": np.mean(F1s.mean(axis=0)),
    "HMean of Averages": hmean(F1s.mean(axis=0)),
    "Avg of HMeans": np.mean([hmean(fold) for fold in F1s]),
    # Teammate's scoring method added:
    "Double HMean": hmean(hmean(F1s, axis=0))
}

# ========================
# UPDATED COMPARISON (WITH BETTER ORDERING)
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<30} | {:>15} | {:>15}".format("Metric", "Your Approach", "Ahmed's Adjusted"))
print("-" * 70)

# Define the preferred order of metrics
preferred_order = [
    "Sex_F F1",
    "ADHD F1",
    "Normal Mean", 
    "HMean of Averages",
    "Avg of HMeans",
    "Double HMean"
]

# Print metrics in the specified order
for key in preferred_order:
    your_val = your_adjusted_results.get(key, "-")
    ahmed_val = ahmed_adjusted_results.get(key, "-")
    
    if isinstance(your_val, float):
        your_val = f"{your_val:.4f}"
    if isinstance(ahmed_val, float):
        ahmed_val = f"{ahmed_val:.4f}"
    
    print("{:<30} | {:>15} | {:>15}".format(key, your_val, ahmed_val))


=== YOUR APPROACH (ADJUSTED) ===


=== AHMED'S APPROACH (ADJUSTED) ===


=== FINAL COMPARISON ===
Metric                         |   Your Approach | Ahmed's Adjusted
----------------------------------------------------------------------
Sex_F F1                       |          0.7010 |          0.6062
ADHD F1                        |          0.8828 |          0.8615
Normal Mean                    |          0.7919 |          0.7339
HMean of Averages              |          0.7814 |          0.7116
Avg of HMeans                  |          0.7812 |          0.7112
Double HMean                   |          0.7811 |          0.7110


In [ ]:
# Constants
SEED = 7
N_SPLITS = 5
target_cols = ['ADHD_Outcome', 'Sex_F']

# Feature selection
non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in target_cols
]
# ========================
# YOUR APPROACH (ADJUSTED)
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

# [Previous initialization code remains the same...]

# Key fix: Use the same features for stratification as for training
full_features = [col for col in train.columns if col not in target_cols]

# Cross-validation - stratify on the full features
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores = []
sex_f1_scores = []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    print(f"\n=== Fold {fold} (Adjusted) ===")
    
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1
    
    # --- Sex model: use ALL features ---
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=train_weights)
    sex_pred = model_sex.predict(X_val[full_features])
    
    # --- ADHD model: use only non-connectome features ---
    model_adhd.fit(X_train[non_connectome_features], y_train_adhd, sample_weight=train_weights)
    adhd_pred = model_adhd.predict(X_val[non_connectome_features])
    
    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Your adjusted results
your_adjusted_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "HMean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Avg of HMeans": np.mean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)]),
    # Teammate's scoring method added:
    "Double HMean": hmean(hmean(np.column_stack([sex_f1_scores, adhd_f1_scores]), axis=0))
}






# ========================
# AHMED'S APPROACH (ADJUSTED)
# ========================
print("\n=== AHMED'S APPROACH (ADJUSTED) ===\n")

# Initialize Ahmed's models with your requested changes
model_sex_ahmed = LogisticRegression(
    penalty='l2', 
    C=10,  # Changed from 0.02 to 10 for sex model
    random_state=SEED,
    solver='saga', 
    max_iter=1000,
    # Removed class_weight='balanced'
)

model_adhd_ahmed = LogisticRegression(
    penalty='l2', 
    C=0.02,  # Kept original C value for ADHD
    random_state=SEED,
    solver='saga', 
    max_iter=1000,
    # Removed class_weight='balanced'
)

# Cross-validation - now stratifying by Sex_F
F1s = []
for fold, (train_idx, val_idx) in enumerate(cv.split(train[non_connectome_features], train['Sex_F']), 1):
    print(f"\n=== Fold {fold} (Ahmed Adjusted) ===")
    
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) + 1
    
    # --- Sex model: use full feature set ---
    full_features = [col for col in train.columns if col not in target_cols]
    model_sex_ahmed.fit(X_train[full_features], X_train['Sex_F'], sample_weight=weights)
    sex_pred = model_sex_ahmed.predict(X_val[full_features])
    
    # --- ADHD model: use non-connectome features ---
    model_adhd_ahmed.fit(X_train[non_connectome_features], X_train['ADHD_Outcome'], sample_weight=weights)
    adhd_pred = model_adhd_ahmed.predict(X_val[non_connectome_features])
    
    # Validation
    valid_idx = X_val[target_cols].notna().all(axis=1)
    sex_f1 = f1_score(X_val.loc[valid_idx, 'Sex_F'], sex_pred[valid_idx])
    adhd_f1 = f1_score(X_val.loc[valid_idx, 'ADHD_Outcome'], adhd_pred[valid_idx])
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    F1s.append([adhd_f1, sex_f1])

F1s = np.array(F1s)

# Adjusted results without Double Harmonic Mean
ahmed_adjusted_results = {
    "Sex_F F1": F1s[:,1].mean(),
    "ADHD F1": F1s[:,0].mean(),
    "Normal Mean": np.mean(F1s.mean(axis=0)),
    "HMean of Averages": hmean(F1s.mean(axis=0)),
    "Avg of HMeans": np.mean([hmean(fold) for fold in F1s]),
    # Teammate's scoring method added:
    "Double HMean": hmean(hmean(F1s, axis=0))
}

# ========================
# UPDATED COMPARISON (WITH BETTER ORDERING)
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<30} | {:>15} | {:>15}".format("Metric", "Your Approach", "Ahmed's Adjusted"))
print("-" * 70)

# Define the preferred order of metrics
preferred_order = [
    "Sex_F F1",
    "ADHD F1",
    "Normal Mean", 
    "HMean of Averages",
    "Avg of HMeans",
    "Double HMean"
]

# Print metrics in the specified order
for key in preferred_order:
    your_val = your_adjusted_results.get(key, "-")
    ahmed_val = ahmed_adjusted_results.get(key, "-")
    
    if isinstance(your_val, float):
        your_val = f"{your_val:.4f}"
    if isinstance(ahmed_val, float):
        ahmed_val = f"{ahmed_val:.4f}"
    
    print("{:<30} | {:>15} | {:>15}".format(key, your_val, ahmed_val))

In [ ]:
# ========================
# YOUR APPROACH (ADJUSTED)
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

# Initialize models
model_sex = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
model_adhd = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# Cross-validation - stratify using ALL features
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores, sex_f1_scores = [], []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Weights
    weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    
    # --- Sex model (all features) ---
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=weights)
    sex_pred = model_sex.predict(X_val[full_features])
    
    # --- ADHD model (non-connectome only) ---
    model_adhd.fit(X_train[non_connectome_features], y_train_adhd, sample_weight=weights)
    adhd_pred = model_adhd.predict(X_val[non_connectome_features])

    # Correct sample weights for validation set
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Results
your_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Double HMean": hmean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)])
}

# ========================
# AHMED'S APPROACH (FIXED)
# ========================
print("\n=== AHMED'S APPROACH (FIXED) ===\n")

# Initialize models (same as yours)
model_sex_ahmed = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
model_adhd_ahmed = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# Cross-validation - identical splitting
F1s = []
for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) + 1
    
    # --- Sex model (all features) ---
    model_sex_ahmed.fit(X_train[full_features], X_train['Sex_F'], sample_weight=weights)
    sex_pred = model_sex_ahmed.predict(X_val[full_features])
    
    # --- ADHD model (non-connectome only) ---
    model_adhd_ahmed.fit(X_train[non_connectome_features], X_train['ADHD_Outcome'], sample_weight=weights)
    adhd_pred = model_adhd_ahmed.predict(X_val[non_connectome_features])
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

    # Scores
    F1s.append([
        f1_score(X_val['ADHD_Outcome'], adhd_pred, sample_weight=val_weights),
        f1_score(X_val['Sex_F'], sex_pred, sample_weight=val_weights)
    ])
    

F1s = np.array(F1s)
ahmed_results = {
    "Sex_F F1": F1s[:,1].mean(),
    "ADHD F1": F1s[:,0].mean(),
    "Double HMean": hmean(hmean(F1s, axis=0))
}

# ========================
# FINAL COMPARISON
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<15} | {:>12} | {:>12}".format("Metric", "Your", "Ahmed's"))
print("-" * 45)
for metric in ["Sex_F F1", "ADHD F1", "Double HMean"]:
    print("{:<15} | {:>12.4f} | {:>12.4f}".format(
        metric, 
        your_results[metric], 
        ahmed_results[metric]
    ))


=== YOUR APPROACH (ADJUSTED) ===


=== AHMED'S APPROACH (FIXED) ===


=== FINAL COMPARISON ===
Metric          |         Your |      Ahmed's
---------------------------------------------
Sex_F F1        |       0.7010 |       0.7010
ADHD F1         |       0.8828 |       0.8828
Double HMean    |       0.7811 |       0.7811


#### Changing seed to see if CV is overfitting or consistent results

In [ ]:
# ========================
# YOUR APPROACH (ADJUSTED)
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

SEED = 42
# Initialize models
model_sex = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
model_adhd = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# Cross-validation - stratify using ALL features
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores, sex_f1_scores = [], []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Weights
    weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    
    # --- Sex model (all features) ---
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=weights)
    sex_pred = model_sex.predict(X_val[full_features])
    
    # --- ADHD model (non-connectome only) ---
    model_adhd.fit(X_train[non_connectome_features], y_train_adhd, sample_weight=weights)
    adhd_pred = model_adhd.predict(X_val[non_connectome_features])

    # Correct sample weights for validation set
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Results
your_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Double HMean": hmean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)])
}

# ========================
# AHMED'S APPROACH (FIXED)
# ========================
print("\n=== AHMED'S APPROACH (FIXED) ===\n")

# Initialize models (same as yours)
model_sex_ahmed = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
model_adhd_ahmed = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# Cross-validation - identical splitting
F1s = []
for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) + 1
    
    # --- Sex model (all features) ---
    model_sex_ahmed.fit(X_train[full_features], X_train['Sex_F'], sample_weight=weights)
    sex_pred = model_sex_ahmed.predict(X_val[full_features])
    
    # --- ADHD model (non-connectome only) ---
    model_adhd_ahmed.fit(X_train[non_connectome_features], X_train['ADHD_Outcome'], sample_weight=weights)
    adhd_pred = model_adhd_ahmed.predict(X_val[non_connectome_features])
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

    # Scores
    F1s.append([
        f1_score(X_val['ADHD_Outcome'], adhd_pred, sample_weight=val_weights),
        f1_score(X_val['Sex_F'], sex_pred, sample_weight=val_weights)
    ])
    

F1s = np.array(F1s)
ahmed_results = {
    "Sex_F F1": F1s[:,1].mean(),
    "ADHD F1": F1s[:,0].mean(),
    "Double HMean": hmean(hmean(F1s, axis=0))
}

# ========================
# FINAL COMPARISON
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<15} | {:>12} | {:>12}".format("Metric", "Your", "Ahmed's"))
print("-" * 45)
for metric in ["Sex_F F1", "ADHD F1", "Double HMean"]:
    print("{:<15} | {:>12.4f} | {:>12.4f}".format(
        metric, 
        your_results[metric], 
        ahmed_results[metric]
    ))

"""
Normal thresholds: 0.5
=== FINAL COMPARISON ===
Metric          |         Your |      Ahmed's
---------------------------------------------
Sex_F F1        |       0.7010 |       0.7010
ADHD F1         |       0.8828 |       0.8828
Double HMean    |       0.7811 |       0.7811
"""


"""
Thresholds: 0.3 sex, 0.4 adhd
=== FINAL COMPARISON ===
Metric          |         Your |      Ahmed's
---------------------------------------------
Sex_F F1        |       0.7088 |       0.7088
ADHD F1         |       0.8801 |       0.8801
Double HMean    |       0.7848 |       0.7848
"""


=== YOUR APPROACH (ADJUSTED) ===


=== AHMED'S APPROACH (FIXED) ===


=== FINAL COMPARISON ===
Metric          |         Your |      Ahmed's
---------------------------------------------
Sex_F F1        |       0.7088 |       0.7088
ADHD F1         |       0.8801 |       0.8801
Double HMean    |       0.7848 |       0.7848


"\n=== FINAL COMPARISON ===\nMetric          |         Your |      Ahmed's\n---------------------------------------------\nSex_F F1        |       0.7010 |       0.7010\nADHD F1         |       0.8828 |       0.8828\nDouble HMean    |       0.7811 |       0.7811\n"

#### Threshold: Sex 0.3 all data, ADHD 0.4 only connectome data

In [ ]:
# ========================
# YOUR APPROACH (ADJUSTED) WITH THRESHOLDS
# ========================
print("\n=== YOUR APPROACH (ADJUSTED) ===\n")

SEED = 42
# Initialize models
model_sex = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
model_adhd = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# Thresholds
SEX_THRESHOLD = 0.3
ADHD_THRESHOLD = 0.4

# Cross-validation - stratify using ALL features
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores, sex_f1_scores = [], []

for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
    # Data prep
    X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex, y_val_sex = X_train['Sex_F'], X_val['Sex_F']
    y_train_adhd, y_val_adhd = X_train['ADHD_Outcome'], X_val['ADHD_Outcome']
    
    # Weights
    weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    
    # --- Sex model (all features) ---
    model_sex.fit(X_train[full_features], y_train_sex, sample_weight=weights)
    sex_proba = model_sex.predict_proba(X_val[full_features])[:, 1]  # Get probabilities
    sex_pred = (sex_proba >= SEX_THRESHOLD).astype(int)  # Apply threshold
    
    # --- ADHD model (non-connectome only) ---
    model_adhd.fit(X_train[non_connectome_features], y_train_adhd, sample_weight=weights)
    adhd_proba = model_adhd.predict_proba(X_val[non_connectome_features])[:, 1]  # Get probabilities
    adhd_pred = (adhd_proba >= ADHD_THRESHOLD).astype(int)  # Apply threshold

    # Validation weights
    val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

    # Calculate scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Results
your_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Double HMean": hmean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)])
}


# # ========================
# # AHMED'S APPROACH (FIXED) WITH THRESHOLDS
# # ========================
# print("\n=== AHMED'S APPROACH (FIXED) ===\n")

# # Initialize models (same as yours)
# model_sex_ahmed = LogisticRegression(penalty='l2', C=10, random_state=SEED, solver='saga', max_iter=1000)
# model_adhd_ahmed = LogisticRegression(penalty='l2', C=0.02, random_state=SEED, solver='saga', max_iter=1000)

# # Cross-validation - identical splitting
# F1s = []
# for fold, (train_idx, val_idx) in enumerate(cv.split(train[full_features], train['Sex_F']), 1):
#     # Data prep
#     X_train, X_val = train.iloc[train_idx], train.iloc[val_idx]
#     weights = ((X_train['Sex_F'] == 1) & (X_train['ADHD_Outcome'] == 1)).astype(int) + 1
    
#     # --- Sex model (all features) ---
#     model_sex_ahmed.fit(X_train[full_features], X_train['Sex_F'], sample_weight=weights)
#     sex_proba = model_sex_ahmed.predict_proba(X_val[full_features])[:, 1]
#     sex_pred = (sex_proba >= SEX_THRESHOLD).astype(int)
    
#     # --- ADHD model (non-connectome only) ---
#     model_adhd_ahmed.fit(X_train[non_connectome_features], X_train['ADHD_Outcome'], sample_weight=weights)
#     adhd_proba = model_adhd_ahmed.predict_proba(X_val[non_connectome_features])[:, 1]
#     adhd_pred = (adhd_proba >= ADHD_THRESHOLD).astype(int)
    
#     # Validation weights
#     val_weights = ((X_val['Sex_F'] == 1) & (X_val['ADHD_Outcome'] == 1)).astype(int) + 1

#     # Scores
#     F1s.append([
#         f1_score(X_val['ADHD_Outcome'], adhd_pred, sample_weight=val_weights),
#         f1_score(X_val['Sex_F'], sex_pred, sample_weight=val_weights)
#     ])

# F1s = np.array(F1s)
# ahmed_results = {
#     "Sex_F F1": F1s[:,1].mean(),
#     "ADHD F1": F1s[:,0].mean(),
#     "Double HMean": hmean(hmean(F1s, axis=0))
# }

# ========================
# FINAL COMPARISON
# ========================
print("\n=== FINAL COMPARISON ===")
print("{:<15} | {:>12} ".format("Metric", "Result"))
# print("{:<15} | {:>12} | {:>12}".format("Metric", "Your", "Ahmed's"))
print("-" * 45)
for metric in ["Sex_F F1", "ADHD F1", "Double HMean"]:
    print("{:<15} | {:>12.4f} | {:>12.4f}".format(
        metric, 
        your_results[metric]
        # ahmed_results[metric]
    ))


=== YOUR APPROACH (ADJUSTED) ===


=== FINAL COMPARISON ===
Metric          |       Result 
---------------------------------------------


IndexError: Replacement index 2 out of range for positional args tuple

### All data including fMRI

In [ ]:
# ========================
# YOUR APPROACH 
# ========================
print("\n=== YOUR APPROACH ===\n")

# Initialize your models
model_sex = LogisticRegression(
    penalty='l2', C=10, random_state=SEED,
    solver='saga', max_iter=1000
)

model_adhd = LogisticRegression(
    penalty='l2', C=0.02, random_state=SEED,
    solver='saga', max_iter=1000
)

# Cross-validation
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores = []
sex_f1_scores = []

# Features for interaction
interaction_features = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity",
    "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score',
    'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems',
    'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'
]

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['Sex_F']), 1):
    print(f"\n=== Fold {fold} ===")
    
    # Split data
    X_train_full, X_val_full = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex = X_train_full['Sex_F']
    y_val_sex = X_val_full['Sex_F']
    y_train_adhd = X_train_full['ADHD_Outcome']
    y_val_adhd = X_val_full['ADHD_Outcome']
    
    # Weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1

    # --- Sex model: use full feature set ---
    sex_features = [col for col in train.columns if col not in target_cols]
    model_sex.fit(X_train_full[sex_features], y_train_sex, sample_weight=train_weights)
    sex_proba_train = model_sex.predict_proba(X_train_full[sex_features])[:, 1]
    sex_proba_val = model_sex.predict_proba(X_val_full[sex_features])[:, 1]
    
    # --- ADHD model: use non-connectome features only ---
    X_train_adhd = X_train_full[non_connectome_features].copy()
    X_val_adhd = X_val_full[non_connectome_features].copy()
    X_train_adhd['sex_proba'] = sex_proba_train
    X_val_adhd['sex_proba'] = sex_proba_val

    for col in interaction_features:
        if col in X_train_adhd.columns:
            X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
            X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']
    
    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)
    
    # Predictions
    sex_pred = model_sex.predict(X_val_full[sex_features])
    adhd_pred = model_adhd.predict(X_val_adhd)
    
    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Results dictionary stays the same
your_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "HMean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Avg of HMeans": np.mean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)])
}


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# Constants
SEED = 7
N_SPLITS = 5


# Initialize models
model_sex = LogisticRegression(
    penalty='l2',
    C=10,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Cross-validation setup
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Score containers
adhd_f1_scores = []
sex_f1_scores = []

# Features used in interaction
interaction_features = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
    "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score', 
    'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems', 
    'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'
] 

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['Sex_F']), 1):
    print(f"\n=== Fold {fold} ===")

    X_train_full, X_val_full = train.iloc[train_idx], train.iloc[val_idx]
    
    # Targets
    y_train_sex = X_train_full['Sex_F']
    y_val_sex = X_val_full['Sex_F']
    y_train_adhd = X_train_full['ADHD_Outcome']
    y_val_adhd = X_val_full['ADHD_Outcome']

    # Sample weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1

    # Use all features except the target columns
    feature_cols = [col for col in train.columns if col not in ['Sex_F', 'ADHD_Outcome']]
    X_train = X_train_full[feature_cols].copy()
    X_val = X_val_full[feature_cols].copy()


    # --- Sex Model ---
    model_sex.fit(X_train, y_train_sex, sample_weight=train_weights)
    sex_train_proba = model_sex.predict_proba(X_train)[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val)[:, 1]

    # --- ADHD Model ---
    X_train_adhd = X_train.copy()
    X_val_adhd = X_val.copy()

    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba

    # Add interaction features
    for col in interaction_features:
        if col in X_train_adhd.columns:
            X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
            X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)

    # Predict
    sex_pred = model_sex.predict(X_val)
    adhd_pred = model_adhd.predict(X_val_adhd)

    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Final results
print("\n=== Competition Results ===")
print(f"Average Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Average ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# NEW: Calculate harmonic mean per fold first
per_fold_hmeans = [hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)]
final_score_hmean_foldwise = np.mean(per_fold_hmeans)

# Original approach for comparison
final_score_mean = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
final_score_hmean = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])

print(f"\nCompetition Score (Normal Mean): {final_score_mean:.4f}")
print(f"Competition Score (Harmonic Mean of Averages): {final_score_hmean:.4f}")
print(f"Competition Score (Average of Harmonic Means): {final_score_hmean_foldwise:.4f}")  # This is likely what leaderboard uses



""" 
=== Fold 1 ===
Sex_F F1: 0.7221
ADHD_Outcome F1: 0.4551

=== Fold 2 ===
Sex_F F1: 0.6894
ADHD_Outcome F1: 0.4519

=== Fold 3 ===
Sex_F F1: 0.7251
ADHD_Outcome F1: 0.4184

=== Fold 4 ===
Sex_F F1: 0.6783
ADHD_Outcome F1: 0.4771

=== Fold 5 ===
Sex_F F1: 0.6901
ADHD_Outcome F1: 0.4966

=== Competition Results ===
Average Sex_F F1: 0.7010 ± 0.0189
Average ADHD F1: 0.4598 ± 0.0263

Competition Score (Normal Mean): 0.5804
Competition Score (Harmonic Mean of Averages): 0.5554
Competition Score (Average of Harmonic Means): 0.5545
"""



=== Fold 1 ===
Sex_F F1: 0.7221
ADHD_Outcome F1: 0.4551

=== Fold 2 ===
Sex_F F1: 0.6894
ADHD_Outcome F1: 0.4519

=== Fold 3 ===
Sex_F F1: 0.7251
ADHD_Outcome F1: 0.4184

=== Fold 4 ===
Sex_F F1: 0.6783
ADHD_Outcome F1: 0.4771

=== Fold 5 ===
Sex_F F1: 0.6901
ADHD_Outcome F1: 0.4966

=== Competition Results ===
Average Sex_F F1: 0.7010 ± 0.0189
Average ADHD F1: 0.4598 ± 0.0263

Competition Score (Normal Mean): 0.5804
Competition Score (Harmonic Mean of Averages): 0.5554
Competition Score (Average of Harmonic Means): 0.5545


### Submission

In [ ]:
# --- Final Training for Your Approach ---
print("\n=== Creating Submission for Your Approach ===")

# Get all features except targets
full_features = [col for col in train.columns if col not in target_cols]

# Train sex model on all features
model_sex.fit(train[full_features], train['Sex_F'])
sex_test_proba = model_sex.predict_proba(test[full_features])[:, 1]

# Train ADHD model on non-connectome features only
model_adhd.fit(train[non_connectome_features], train['ADHD_Outcome'])

# Make predictions
sex_pred_test = model_sex.predict(test[full_features])
adhd_pred_test = model_adhd.predict(test[non_connectome_features])

# Create submission
submission_yours = test.reset_index()[['participant_id']].copy()
submission_yours['Sex_F'] = sex_pred_test
submission_yours['ADHD_Outcome'] = adhd_pred_test

# Save
submission_yours.to_csv('YourApproach_Submission.csv', index=False)
print("✅ Your approach submission saved")


=== Creating Submission for Your Approach ===
✅ Your approach submission saved


In [ ]:
# --- Final Training for Ahmed's Approach ---
print("\n=== Creating Submission for Ahmed's Approach ===")

# Re-initialize models with Ahmed's parameters
model_sex_ahmed = LogisticRegression(
    penalty='l2', C=10, random_state=SEED,
    solver='saga', max_iter=1000
)

model_adhd_ahmed = LogisticRegression(
    penalty='l2', C=0.02, random_state=SEED,
    solver='saga', max_iter=1000
)

# Train sex model on all features
model_sex_ahmed.fit(train[full_features], train['Sex_F'])

# Train ADHD model on non-connectome features
model_adhd_ahmed.fit(train[non_connectome_features], train['ADHD_Outcome'])

# Make predictions
sex_pred_test = model_sex_ahmed.predict(test[full_features])
adhd_pred_test = model_adhd_ahmed.predict(test[non_connectome_features])

# Create submission
submission_ahmed = test.reset_index()[['participant_id']].copy()
submission_ahmed['Sex_F'] = sex_pred_test
submission_ahmed['ADHD_Outcome'] = adhd_pred_test

# Save
submission_ahmed.to_csv('AhmedApproach_Submission.csv', index=False)
print("✅ Ahmed's approach submission saved")


=== Creating Submission for Ahmed's Approach ===
✅ Ahmed's approach submission saved


In [ ]:
import pandas as pd

# Get feature names and coefficients
feature_names = X_train_adhd.columns
coefficients = model_adhd.coef_[0]

# Create DataFrame for feature importance
importance_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients
})

# Sort by coefficient value (descending: positive to negative)
importance_df_sorted = importance_df.sort_values(by='coefficient', ascending=False)

# Print sorted features
print("\n=== Feature Importances (Sorted by Coefficient Sign) ===")
print(importance_df_sorted.to_string(index=False))



=== Feature Importances (Sorted by Coefficient Sign) ===
                         feature  coefficient
           SDQ_SDQ_Hyperactivity     0.516202
           SDQ_SDQ_Externalizing     0.299540
       SDQ_SDQ_Generating_Impact     0.267966
     SDQ_SDQ_Hyperactivity_x_sex     0.203472
      SDQ_SDQ_Difficulties_Total     0.179835
 SDQ_SDQ_Generating_Impact_x_sex     0.077879
                  APQ_P_APQ_P_PM     0.047455
               SDQ_SDQ_Prosocial     0.040775
         SDQ_SDQ_Prosocial_x_sex     0.027435
           SDQ_SDQ_Peer_Problems     0.025528
            female_symptom_score     0.012339
      female_symptom_score_x_sex    -0.009089
      ColorVision_CV_Score_x_sex    -0.015238
                 APQ_P_APQ_P_OPD    -0.016949
           SDQ_SDQ_Internalizing    -0.018389
                  APQ_P_APQ_P_ID    -0.021111
     SDQ_SDQ_Internalizing_x_sex    -0.029022
                       sex_proba    -0.038428
                  APQ_P_APQ_P_PP    -0.038470
           APQ_P_APQ_P

In [ ]:
# --- Final Training on Full Data ---
X_full = train[non_connectome_features].copy()
y_sex = train['Sex_F']
y_adhd = train['ADHD_Outcome']

# Train sex model on full data
model_sex.fit(X_full, y_sex)
sex_test_proba = model_sex.predict_proba(test[non_connectome_features])[:, 1]

# Prepare test data
X_test = test[non_connectome_features].copy()
X_test['sex_proba'] = sex_test_proba

# Add interaction features to test set
for col in interaction_features:
    if col in X_test.columns:
        X_test[f'{col}_x_sex'] = X_test[col] * X_test['sex_proba']

# Train ADHD model on full data with interaction features
X_full_adhd = X_full.copy()
X_full_adhd['sex_proba'] = model_sex.predict_proba(X_full)[:, 1]

for col in interaction_features:
    if col in X_full_adhd.columns:
        X_full_adhd[f'{col}_x_sex'] = X_full_adhd[col] * X_full_adhd['sex_proba']

model_adhd.fit(X_full_adhd, y_adhd)

# Make predictions
sex_pred_test = np.ones(len(test), dtype=int)
adhd_pred_test = model_adhd.predict(X_test)

# Create submission DataFrame
submission = test.reset_index()[['participant_id']].copy()
submission['Sex_F'] = sex_pred_test
submission['ADHD_Outcome'] = adhd_pred_test

# Save submission
submission.to_csv('NormalMean6966HMean6503Sex1.csv', index=False)
print("\n✅ Submission saved to 'NormalMean6966HMean6503.csv'")



✅ Submission saved to 'NormalMean6966HMean6503.csv'


### All data including fMRI

In [ ]:
# ========================
# YOUR APPROACH 
# ========================
print("\n=== YOUR APPROACH ===\n")

# Initialize your models
model_sex = LogisticRegression(
    penalty='l2', C=10, random_state=SEED,
    solver='saga', max_iter=1000
)

model_adhd = LogisticRegression(
    penalty='l2', C=0.02, random_state=SEED,
    solver='saga', max_iter=1000
)

# Cross-validation
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
adhd_f1_scores = []
sex_f1_scores = []

# Features for interaction
interaction_features = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity",
    "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score',
    'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems',
    'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'
]

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['Sex_F']), 1):
    print(f"\n=== Fold {fold} ===")
    
    # Split data
    X_train_full, X_val_full = train.iloc[train_idx], train.iloc[val_idx]
    y_train_sex = X_train_full['Sex_F']
    y_val_sex = X_val_full['Sex_F']
    y_train_adhd = X_train_full['ADHD_Outcome']
    y_val_adhd = X_val_full['ADHD_Outcome']
    
    # Weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1

    # --- Sex model: use full feature set ---
    sex_features = [col for col in train.columns if col not in target_cols]
    model_sex.fit(X_train_full[sex_features], y_train_sex, sample_weight=train_weights)
    sex_proba_train = model_sex.predict_proba(X_train_full[sex_features])[:, 1]
    sex_proba_val = model_sex.predict_proba(X_val_full[sex_features])[:, 1]
    
    # --- ADHD model: use non-connectome features only ---
    X_train_adhd = X_train_full[non_connectome_features].copy()
    X_val_adhd = X_val_full[non_connectome_features].copy()
    X_train_adhd['sex_proba'] = sex_proba_train
    X_val_adhd['sex_proba'] = sex_proba_val

    for col in interaction_features:
        if col in X_train_adhd.columns:
            X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
            X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']
    
    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)
    
    # Predictions
    sex_pred = model_sex.predict(X_val_full[sex_features])
    adhd_pred = model_adhd.predict(X_val_adhd)
    
    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)
    
    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")
    
    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Results dictionary stays the same
your_results = {
    "Sex_F F1": np.mean(sex_f1_scores),
    "ADHD F1": np.mean(adhd_f1_scores),
    "Normal Mean": np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "HMean of Averages": hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)]),
    "Avg of HMeans": np.mean([hmean([s, a]) for s, a in zip(sex_f1_scores, adhd_f1_scores)])
}


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import hmean

# Constants
SEED = 7
N_SPLITS = 5


# Initialize models
model_sex = LogisticRegression(
    penalty='l2',
    C=10,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

model_adhd = LogisticRegression(
    penalty='l2',
    C=0.02,
    random_state=SEED,
    solver='saga',
    max_iter=1000,
)

# Cross-validation setup
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

# Score containers
adhd_f1_scores = []
sex_f1_scores = []

# Features used in interaction
interaction_features = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
    "SDQ_SDQ_Generating_Impact", 'female_symptom_score', 'male_symptom_score', 
    'ColorVision_CV_Score', 'SDQ_SDQ_Emotional_Problems', 
    'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Internalizing'
] 

for fold, (train_idx, val_idx) in enumerate(cv.split(train, train['Sex_F']), 1):
    print(f"\n=== Fold {fold} ===")

    X_train_full, X_val_full = train.iloc[train_idx], train.iloc[val_idx]
    
    # Targets
    y_train_sex = X_train_full['Sex_F']
    y_val_sex = X_val_full['Sex_F']
    y_train_adhd = X_train_full['ADHD_Outcome']
    y_val_adhd = X_val_full['ADHD_Outcome']

    # Sample weights
    train_weights = ((y_train_sex == 1) & (y_train_adhd == 1)).astype(int) + 1
    val_weights = ((y_val_sex == 1) & (y_val_adhd == 1)).astype(int) + 1

    # Use all features except the target columns
    feature_cols = [col for col in train.columns if col not in ['Sex_F', 'ADHD_Outcome']]
    X_train = X_train_full[feature_cols].copy()
    X_val = X_val_full[feature_cols].copy()


    # --- Sex Model ---
    model_sex.fit(X_train, y_train_sex, sample_weight=train_weights)
    sex_train_proba = model_sex.predict_proba(X_train)[:, 1]
    sex_val_proba = model_sex.predict_proba(X_val)[:, 1]

    # --- ADHD Model ---
    X_train_adhd = X_train.copy()
    X_val_adhd = X_val.copy()

    X_train_adhd['sex_proba'] = sex_train_proba
    X_val_adhd['sex_proba'] = sex_val_proba

    # Add interaction features
    for col in interaction_features:
        if col in X_train_adhd.columns:
            X_train_adhd[f'{col}_x_sex'] = X_train_adhd[col] * X_train_adhd['sex_proba']
            X_val_adhd[f'{col}_x_sex'] = X_val_adhd[col] * X_val_adhd['sex_proba']

    model_adhd.fit(X_train_adhd, y_train_adhd, sample_weight=train_weights)

    # Predict
    sex_pred = model_sex.predict(X_val)
    adhd_pred = model_adhd.predict(X_val_adhd)

    # Scores
    sex_f1 = f1_score(y_val_sex, sex_pred, sample_weight=val_weights)
    adhd_f1 = f1_score(y_val_adhd, adhd_pred, sample_weight=val_weights)

    print(f"Sex_F F1: {sex_f1:.4f}")
    print(f"ADHD_Outcome F1: {adhd_f1:.4f}")

    sex_f1_scores.append(sex_f1)
    adhd_f1_scores.append(adhd_f1)

# Final results
print("\n=== Competition Results ===")
print(f"Average Sex_F F1: {np.mean(sex_f1_scores):.4f} ± {np.std(sex_f1_scores):.4f}")
print(f"Average ADHD F1: {np.mean(adhd_f1_scores):.4f} ± {np.std(adhd_f1_scores):.4f}")

# NEW: Calculate harmonic mean per fold first
per_fold_hmeans = [hmean([sex, adhd]) for sex, adhd in zip(sex_f1_scores, adhd_f1_scores)]
final_score_hmean_foldwise = np.mean(per_fold_hmeans)

# Original approach for comparison
final_score_mean = np.mean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])
final_score_hmean = hmean([np.mean(sex_f1_scores), np.mean(adhd_f1_scores)])

print(f"\nCompetition Score (Normal Mean): {final_score_mean:.4f}")
print(f"Competition Score (Harmonic Mean of Averages): {final_score_hmean:.4f}")
print(f"Competition Score (Average of Harmonic Means): {final_score_hmean_foldwise:.4f}")  # This is likely what leaderboard uses



""" 
=== Fold 1 ===
Sex_F F1: 0.7221
ADHD_Outcome F1: 0.4551

=== Fold 2 ===
Sex_F F1: 0.6894
ADHD_Outcome F1: 0.4519

=== Fold 3 ===
Sex_F F1: 0.7251
ADHD_Outcome F1: 0.4184

=== Fold 4 ===
Sex_F F1: 0.6783
ADHD_Outcome F1: 0.4771

=== Fold 5 ===
Sex_F F1: 0.6901
ADHD_Outcome F1: 0.4966

=== Competition Results ===
Average Sex_F F1: 0.7010 ± 0.0189
Average ADHD F1: 0.4598 ± 0.0263

Competition Score (Normal Mean): 0.5804
Competition Score (Harmonic Mean of Averages): 0.5554
Competition Score (Average of Harmonic Means): 0.5545
"""



=== Fold 1 ===
Sex_F F1: 0.7221
ADHD_Outcome F1: 0.4551

=== Fold 2 ===
Sex_F F1: 0.6894
ADHD_Outcome F1: 0.4519

=== Fold 3 ===
Sex_F F1: 0.7251
ADHD_Outcome F1: 0.4184

=== Fold 4 ===
Sex_F F1: 0.6783
ADHD_Outcome F1: 0.4771

=== Fold 5 ===
Sex_F F1: 0.6901
ADHD_Outcome F1: 0.4966

=== Competition Results ===
Average Sex_F F1: 0.7010 ± 0.0189
Average ADHD F1: 0.4598 ± 0.0263

Competition Score (Normal Mean): 0.5804
Competition Score (Harmonic Mean of Averages): 0.5554
Competition Score (Average of Harmonic Means): 0.5545


### Submission

In [79]:
# --- Final Training for Your Approach ---
print("\n=== Creating Submission for Your Approach ===")

# Get all features except targets
full_features = [col for col in train.columns if col not in target_cols]

# Train sex model on all features
model_sex.fit(train[full_features], train['Sex_F'])
sex_test_proba = model_sex.predict_proba(test[full_features])[:, 1]

# Train ADHD model on non-connectome features only
model_adhd.fit(train[non_connectome_features], train['ADHD_Outcome'])

# Make predictions
sex_pred_test = model_sex.predict(test[full_features])
adhd_pred_test = model_adhd.predict(test[non_connectome_features])

# Create submission
submission_yours = test.reset_index()[['participant_id']].copy()
submission_yours['Sex_F'] = sex_pred_test
submission_yours['ADHD_Outcome'] = adhd_pred_test

# Save
submission_yours.to_csv('YourApproach_Submission.csv', index=False)
print("✅ Your approach submission saved")


=== Creating Submission for Your Approach ===
✅ Your approach submission saved


In [80]:
# --- Final Training for Ahmed's Approach ---
print("\n=== Creating Submission for Ahmed's Approach ===")

# Re-initialize models with Ahmed's parameters
model_sex_ahmed = LogisticRegression(
    penalty='l2', C=10, random_state=SEED,
    solver='saga', max_iter=1000
)

model_adhd_ahmed = LogisticRegression(
    penalty='l2', C=0.02, random_state=SEED,
    solver='saga', max_iter=1000
)

# Train sex model on all features
model_sex_ahmed.fit(train[full_features], train['Sex_F'])

# Train ADHD model on non-connectome features
model_adhd_ahmed.fit(train[non_connectome_features], train['ADHD_Outcome'])

# Make predictions
sex_pred_test = model_sex_ahmed.predict(test[full_features])
adhd_pred_test = model_adhd_ahmed.predict(test[non_connectome_features])

# Create submission
submission_ahmed = test.reset_index()[['participant_id']].copy()
submission_ahmed['Sex_F'] = sex_pred_test
submission_ahmed['ADHD_Outcome'] = adhd_pred_test

# Save
submission_ahmed.to_csv('AhmedApproach_Submission.csv', index=False)
print("✅ Ahmed's approach submission saved")


=== Creating Submission for Ahmed's Approach ===
✅ Ahmed's approach submission saved


In [ ]:
import pandas as pd

# Get feature names and coefficients
feature_names = X_train_adhd.columns
coefficients = model_adhd.coef_[0]

# Create DataFrame for feature importance
importance_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients
})

# Sort by coefficient value (descending: positive to negative)
importance_df_sorted = importance_df.sort_values(by='coefficient', ascending=False)

# Print sorted features
print("\n=== Feature Importances (Sorted by Coefficient Sign) ===")
print(importance_df_sorted.to_string(index=False))



=== Feature Importances (Sorted by Coefficient Sign) ===
                         feature  coefficient
           SDQ_SDQ_Hyperactivity     0.516202
           SDQ_SDQ_Externalizing     0.299540
       SDQ_SDQ_Generating_Impact     0.267966
     SDQ_SDQ_Hyperactivity_x_sex     0.203472
      SDQ_SDQ_Difficulties_Total     0.179835
 SDQ_SDQ_Generating_Impact_x_sex     0.077879
                  APQ_P_APQ_P_PM     0.047455
               SDQ_SDQ_Prosocial     0.040775
         SDQ_SDQ_Prosocial_x_sex     0.027435
           SDQ_SDQ_Peer_Problems     0.025528
            female_symptom_score     0.012339
      female_symptom_score_x_sex    -0.009089
      ColorVision_CV_Score_x_sex    -0.015238
                 APQ_P_APQ_P_OPD    -0.016949
           SDQ_SDQ_Internalizing    -0.018389
                  APQ_P_APQ_P_ID    -0.021111
     SDQ_SDQ_Internalizing_x_sex    -0.029022
                       sex_proba    -0.038428
                  APQ_P_APQ_P_PP    -0.038470
           APQ_P_APQ_P

In [56]:
# --- Final Training on Full Data ---
X_full = train[non_connectome_features].copy()
y_sex = train['Sex_F']
y_adhd = train['ADHD_Outcome']

# Train sex model on full data
model_sex.fit(X_full, y_sex)
sex_test_proba = model_sex.predict_proba(test[non_connectome_features])[:, 1]

# Prepare test data
X_test = test[non_connectome_features].copy()
X_test['sex_proba'] = sex_test_proba

# Add interaction features to test set
for col in interaction_features:
    if col in X_test.columns:
        X_test[f'{col}_x_sex'] = X_test[col] * X_test['sex_proba']

# Train ADHD model on full data with interaction features
X_full_adhd = X_full.copy()
X_full_adhd['sex_proba'] = model_sex.predict_proba(X_full)[:, 1]

for col in interaction_features:
    if col in X_full_adhd.columns:
        X_full_adhd[f'{col}_x_sex'] = X_full_adhd[col] * X_full_adhd['sex_proba']

model_adhd.fit(X_full_adhd, y_adhd)

# Make predictions
sex_pred_test = np.ones(len(test), dtype=int)
adhd_pred_test = model_adhd.predict(X_test)

# Create submission DataFrame
submission = test.reset_index()[['participant_id']].copy()
submission['Sex_F'] = sex_pred_test
submission['ADHD_Outcome'] = adhd_pred_test

# Save submission
submission.to_csv('NormalMean6966HMean6503Sex1.csv', index=False)
print("\n✅ Submission saved to 'NormalMean6966HMean6503.csv'")



✅ Submission saved to 'NormalMean6966HMean6503.csv'
